In [ ]:
import numpy as np
import pandas as pd
from scipy import ndimage
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from tables import *
import os
import operator
import sys
import csv
import h5py
import time
from astropy.time import Time
import astropy.units as u
from numba import jit,njit,int32,float32
from numba.typed import List
from functions_import import lin_fit, intersection, plot_scatter, plot_from_pix, draw_box,get_x_y_from_pix
from functions_import import get_nn_pix, plot_lin_fit, read_data_sets,apply_selection_cut
def import_module(filename):
    with open(filename, 'r') as file:
        file_contents = file.read()
    module_object =  exec(file_contents)
    return module_object
import_module("D:\\Masterarbeit ECAP\\Trail finder\\functions_import.py")
typed_nn_pix = List()
for i in range(len(nn_pix)):
    typed_nn_pix_entry = List()
    [typed_nn_pix_entry.append(x) for x in nn_pix[i]]
    typed_nn_pix.append(typed_nn_pix_entry)

In [ ]:
from functions_import import lin_fit, intersection, plot_scatter, plot_from_pix, draw_box,get_x_y_from_pix
from functions_import import get_nn_pix, plot_lin_fit, read_data_sets, apply_selection_cut

In [ ]:
###Initialisation of variable nruns and dictionaries:
#BE CAREFUL NOT TO DELETE EVERYTHING THAT'S READ IN !!!!!
####perhaps not neccessary anymore...########

nruns = []
trails_high_avg = {}
avg_pix = {}
avg_tmp = {}
delta_pixel = {}
high_pixel = {}
roll_avg = {}
zenith = {}
azimuth = {}
broken_pixel = {}
trackruns = {}
possible_meteorites = {}

path = {}


In [ ]:
%%time
#Reading data of new_nruns using .txt files, not overwriting the old ones 
#Use pix ID to get x and y pos

####perhaps not neccessary anymore...########
telId = 5
for nrun in new_nruns:
    high_pixel[nrun] = {}
    high_path = "high_pixel\\"+runs_directory+"\\"+str(nrun)+"\\"
    print(high_path)
    #for telId in range(5,6):#Set to only load in CT5 data, can be adjusted later on   
    try:
        print("Reading high_pix_"+str(nrun)+"_CT_"+str(telId)+".txt")        
        high_pixel[nrun]["pix"+str(telId)] = np.loadtxt(high_path + "high_pix_"+str(nrun)+"_CT_"+str(telId)+".txt", 
                                                        usecols = 0, delimiter = ";")
        high_pixel[nrun][telId] = np.loadtxt(high_path + "high_pix_"+str(nrun)+"_CT_"+str(telId)+".txt", 
                                                        usecols = 1, delimiter = ";")
        high_pixel[nrun]["tmp"+str(telId)] = np.loadtxt(high_path + "high_pix_"+str(nrun)+"_CT_"+str(telId)+".txt", 
                                                        usecols = 2, delimiter = ";")
    except:
        print("No file named high_pix_"+str(nrun)+"_CT_"+str(telId)+".txt found")

In [ ]:
#This only looks for CT5.h5 files, hence no separation for telId 

####already in funtinons_import########
def read_data_sets(firstrun, lastrun):
    if (firstrun%200!=0  or lastrun%200!=0):
        print("Numbers divisible by 200 needed!")
        return
    if firstrun> lastrun:
        print("Second number needs to be larger than first one!")
        return
    run_set = 0
    new_nruns = []
    new_high_pixel = {}
    os.chdir("D:\\Masterarbeit ECAP\\First plots\\eval_data\\high_pixel")
    while (run_set<(lastrun-firstrun)/200):
        runs_directory = "run"+str(firstrun+run_set*200)+"-"+str(firstrun+run_set*200+199)
        try:
            os.chdir("D:\\Masterarbeit ECAP\\First plots\\eval_data\\high_pixel\\"+runs_directory)
        except:
            print("folder","\""+runs_directory+"\"", "does not exist")
            run_set+=1
            continue
        run_numbers = os.listdir(os.curdir)
        for i in range(len(run_numbers)):
            try:
                new_nruns.append(int(run_numbers[i]))
                try:
                    print(os.getcwd())
#                     print(str(run_numbers[i])+"\\high_pix_"+str(run_numbers[i]))#+"_CT_"+str(telId)+".h5")
                    hfile = h5py.File(str(run_numbers[i])+"\\high_pix_"+str(run_numbers[i])+"_CT_"+str(5)+".h5", "r") 
                    print("Reading high_pix_"+str(run_numbers[i])+"_CT_"+str(5)+".h5")
                    new_high_pixel[int(run_numbers[i])] = {}
                    new_high_pixel[int(run_numbers[i])]["pix"] = hfile["Pix ID"]
                    new_high_pixel[int(run_numbers[i])]["brightness"] = hfile["Brightness"]
                    new_high_pixel[int(run_numbers[i])]["time"] = hfile["Time"]
                    hfile.close()
                    print("Reading successful")
                except:
                    print("No file named high_pix_"+str(run_numbers[i])+"_CT_"+str(5)+".h5 found in folder", run_numbers[i])
            except:
                print("No runs in folder/not a folder:", run_numbers[i])
        
        run_set+=1
    return new_high_pixel

In [ ]:
%%time 
test = read_data_sets(157000,175200)

In [ ]:
def apply_selection_cut_from_h5(dict_high_pixel):
    max_counts = 70
    high_pixel_cut = {}
    for nrun in dict_high_pixel.keys():
        high_pixel_cut[nrun] = {}
        print(type(dict_high_pixel[nrun]["pix"]))
        values, counts = np.unique(np.array(dict_high_pixel[nrun]["pix"]), return_counts=True)
        mask_max_counts = counts<max_counts       
        st = set(values[mask_max_counts])
        result = [i for i, e in enumerate(np.array(dict_high_pixel[nrun]["pix"])) if e in st]
        high_pixel_cut[nrun]["pix"] = np.array(dict_high_pixel[nrun]["pix"])[result].astype(int)
        high_pixel_cut[nrun]["brightness"] = np.array(dict_high_pixel[nrun]["brightness"])[result]
        high_pixel_cut[nrun]["time"] = np.array(dict_high_pixel[nrun]["time"])[result]-dict_high_pixel[nrun]["time"][0]
    return high_pixel_cut

In [ ]:
%%time
hfile = h5py.(test[158229]["pix"])

# print(test.keys())
# # test_cut = apply_selection_cut(test)
# print(type(test[158229]["pix"]))

In [ ]:
######NEEDS TO BE DONE SOME TIME##################
# def print_important_params(track_x, track_y, track_z):
#     print("Track", N, end =": ")    
#     start_time = track_z[0]
#     stop_time = track_z[-1]
#     print("Start:", int(start_time), "s , Stop: ", int(stop_time), "s , Duration: ", int(stop_time-start_time), "s" )
#     popt_param, pcov_param = curve_fit(lin_fit, track_x, track_y)
#     print("linear fit : y =",round(popt_param[0],4),"* x +",round(popt_param[1],4)) 
#     x_y_len = np.sqrt((track_x[-1]-track_x[0])**2+
#                       (lin_fit(np.array(track_x), *popt_param)[-1] - 
#                        lin_fit(np.array(track_x), *popt_param)[0])**2 
#                              )
#     print("Length on cam:", round(x_y_len, 4), "m(?)")
#     print("Speed on cam: ", round(x_y_len/dur,4), "m/s")
#     x_y = []
#     for i in range(len(track_x)):
#         x_y.append((round(track_x[i],4), 
#                     round(track_y[i],4)))
#     print("Unique pixels:",len(list(set(x_y))))
#     print("Pixels per unit of length:", round(len(list(set(x_y)))/x_y_len, 4) )   

In [ ]:
@jit 
##################Best Sorter yet##################

def track_sorter(dict_pix, dict_tmp, dict_brightness, typed_nn_pixels):#typed_nn_pixels needed to work together with @jit
    if len(dict_pix)==0:
        tracks = List([[[-1.],[-1.],[-1.]]])
        possible_meteorites = List([[[-1.],[-1.],[-1.]]])
        #It is not possible to have neg. values in track, hence if -1, no track was found
        return tracks, possible_meteorites
    pix = list(dict_pix)
    tmp = list(dict_tmp)
    brightness = list(dict_brightness)    
    nn_pixels = list(typed_nn_pixels)
####Comments are for jit compatible List(), if this is needed in the future######
#     x = List()
#     y = List()
#     z = List()
#     x.append(float(pix[0]))
#     y.append(float(tmp[0]))
#     z.append(float(brightness[0]))
#     track_zero = List()
#     track_zero.append(x)
#     track_zero.append(y)
#     track_zero.append(z)
#     tracks = List()
#     tracks.append(track_zero)
#     x = List(float(pix[0]))
#     y = List(float(tmp[0]))
#     z = List(float(brightness[0]))
#     print(x,y,z)
#     track_zero = List([x,y,z])
#     x_met = List()
#     y_met = List()
#     z_met = List()
#     x_met.append(float(pix[0]))
#     y_met.append(float(tmp[0]))
#     z_met.append(float(brightness[0]))
#     track_zero_met = List()
#     track_zero_met.append(x)
#     track_zero_met.append(y)
#     track_zero_met.append(z)
#     possible_meteorites = List()
#     possible_meteorites.append(track_zero_met)
####Comments are for jit compatible List(), if this is needed in the future######
    tracks = List([ [[float(pix[0])],[float(tmp[0])], [float(brightness[0])]]])
    
#     tracks = List([tracks_zero])
    for i in range(1, len(pix)):
        appended_to_track = False
        for N in range(len(tracks)):
            if tmp[i]-5 < tracks[N][1][-1]:
                if len([x for x in nn_pixels[pix[i]] if x in tracks[N][0][-20:]])>0:
                    tracks[N][0].append(pix[i])
                    tracks[N][1].append(tmp[i])
                    tracks[N][2].append(brightness[i])
                    appended_to_track  = True 
                    break 
        if appended_to_track == False:
            tracks.append([[float(pix[i])],[float(tmp[i])], [float(brightness[i])]])

    possible_meteorites = List([[[0.],[0.],[0.]]])
    to_remove = np.full((1,len(tracks)), -1)[0]   
    for N in range(len(tracks)):        
        if len(np.unique(np.array(tracks[N][0])))<3:
            possible_meteorites.append(tracks[N])
            to_remove[N] = N
        elif len(np.unique(np.array(tracks[N][1])))<3:
            possible_meteorites.append(tracks[N])
            to_remove[N] = N
    del possible_meteorites[0]
    if len(possible_meteorites)==0:
        possible_meteorites = List([[[-1.],[-1.],[-1.]]])
    
    to_remove = to_remove[to_remove != -1]
    for i in range(len(to_remove)):
        del tracks[to_remove[-(i+1)]]
    if len(tracks)==0:
        tracks = List([[[-1.],[-1.],[-1.]]])
    return tracks, possible_meteorites

In [ ]:
# @jit
def adding_possible_meteorites(possible_met):# entry needs to be list of possible meteorite tracks with lists for pix, tmp, brightness
    tmp = list(possible_met)    
    trash = []
    if len(possible_met)==0:
        return tmp, trash
#     for i in range(len(possible_met)):
#         print(possible_met[i])
    if len(tmp) == 0:
        return 
    for N in reversed(range(len(tmp)-1)):
        if len([x for x in tmp[N][1] if x in tmp[N-1][1]])>0:
            tmp[N-1][0] = tmp[N][0]+ tmp[N-1][0]#pix
            tmp[N-1][1] = tmp[N][1]+ tmp[N-1][1]#tmp
            tmp[N-1][2] = tmp[N][2]+ tmp[N-1][2]#brightness
            del tmp[N]
    trash = []
    for N in range(len(tmp)):
        if (len(np.unique(tmp[N][0]))<2 or len(np.unique(tmp[N][1]))>100):
            trash.append(tmp[N])
    return tmp, trash

In [ ]:
@jit 
def adding_possible_meteorites_jit(pos_met):
    tmp = List(pos_met)
    trash = List()
    if (len(pos_met)==0 or len(tmp)==0):
        trash.append([[-1.],[-1.],[-1.]])
        return tmp, trash
    print(len(tmp))
    index_to_del = List()
    for N in range(len(tmp)-1):
        if len([x for x in tmp[-(N+1)][1] if x in tmp[-(N+2)][1]])>0:
#             trash.append(1)
            tmp[-(N+2)][0] = tmp[-(N+2)][0]+ tmp[-(N+1)][0]#pix
            tmp[-(N+2)][1] = tmp[-(N+2)][1]+ tmp[-(N+1)][1]#tmp
            tmp[-(N+2)][2] = tmp[-(N+2)][2]+ tmp[-(N+1)][2]#brightness
            index_to_del.append(-(N+1))
#             del tmp[-(N+1)]
    print(len(tmp))
    for N in range(len(tmp)):
        if (len(np.unique(np.array(tmp[N][0])))<2 or len(np.unique(np.array(tmp[N][1])))>100):
            trash.append(tmp[N])
            print(len(tmp))
            index_to_del.append(N)
#             del tmp[-(N+1)]
    print(index_to_del)
    return tmp, trash


In [ ]:
high_pixel = read_data_sets(175000,175200)
print(high_pixel.keys())
# a,b = adding_possible_meteorites_jit(possible_meteorite)
# for i in range(len(a)):
    
# print(a)
# print(b)

In [ ]:
def plot_poss_met(poss_met):#Input is possible meteorite or track
    if len(poss_met)==0:
        return
    for i in range(len(poss_met)):
        plot_from_pix(poss_met[i][0], poss_met[i][1])
        draw_box()
        #plt.colorbar()
        plt.show()
    return

In [ ]:
def get_lin_fit(track):
    if track[0][0][0] == -1:
        return 
    x_box = np.arange(-1.25, 1.25, 0.1)
    for i in range(len(track)):
        pix_x, pix_y = get_x_y_from_pix(track[i][0])
        popt, pcov = curve_fit(lin_fit, pix_x, pix_y)
        print(popt)
        plt.plot(x_box, lin_fit(x_box, popt[0], popt[1]))
        plt.scatter(pix_x, pix_y)
        plt.xlim(-1.25, 1.25)
        plt.ylim(-1.25, 1.25)
    plt.show
    return

In [ ]:
def get_center_of_gravity(track):
    if track[0][0][0] == -1:
        cog = [[-10,-10]]
        return cog
    cog = []
    for i in range(len(track)):
        pix_x, pix_y = get_x_y_from_pix(track[i][0])
        cog_x = round(np.average(pix_x), 6)
        cog_y = round(np.average(pix_y), 6)
        cog.append([cog_x, cog_y])
    plt.show()
    return cog

In [ ]:
#velocity and direction
def get_velo(track):
    if track[0][0][0] == -1:
        vel = [0]
        return vel
    vel = []
    for i in range(len(track)):
        pix_x, pix_y = get_x_y_from_pix(track[i][0])
        d = np.sqrt((pix_x[-1]-pix_x[0])**2+(pix_y[-1]-pix_y[0])**2)
        t = track[i][1][-1]-track[i][1][0]
        v = round(d/t, 5)
        vel.append(v)
    return vel
def get_direction(track): #get direction from center of gravtiy in polar coordinates
    if track[0][0][0] == -1:
        direction = [[-1, 0]]
        return direction
    direction = []
    cent_of_grav = get_center_of_gravity(track)
    for i in range(len(track)):
        pix_x, pix_y = get_x_y_from_pix(track[i][0])
        direction_vec = [pix_x[-1]-cent_of_grav[i][0],pix_y[-1]-cent_of_grav[i][1]]
        direction_polar_r = np.sqrt(direction_vec[0]**2+direction_vec[1]**2)
        direction_polar_phi = np.arctan(direction_vec[1]/direction_vec[0])*180/np.pi
        direction.append([round(direction_polar_r,6), round(direction_polar_phi,6)])
    return direction

In [ ]:
def get_box():
    mask_x_top = np.around(np.array(flash_geom_x), 6) == np.max(np.around(flash_geom_x, 6))
    mask_x_bot = np.around(np.array(flash_geom_x), 6) == np.min(np.around(flash_geom_x, 6))
    mask_y_top = np.around(np.array(flash_geom_y), 6) == np.max(np.around(flash_geom_y, 6))
    mask_y_bot = np.around(np.array(flash_geom_y), 6) == np.min(np.around(flash_geom_y, 6))
    
    y_secondtolast = sorted(np.unique(np.around(flash_geom_y, 6)))[-2]
    y_secondtofirst = sorted(np.unique(np.around(flash_geom_y, 6)))[1]
    mask1 = np.around(np.array(flash_geom_y), 6) == y_secondtolast
    mask2 = np.around(np.array(flash_geom_y), 6) == y_secondtofirst
    
    box_x = [flash_geom_x[mask_x_top][0],
             np.max(np.around(flash_geom_x[mask1], 6)),
            np.max(flash_geom_x[mask_y_top]),
            np.min(flash_geom_x[mask_y_top]),
            flash_geom_x[mask_x_bot][0],
            flash_geom_x[mask_x_bot][-1],
            np.min(flash_geom_x[mask_y_bot]),
            np.max(flash_geom_x[mask_y_bot]),
             np.max(np.around(flash_geom_x[mask2], 6)),
            flash_geom_x[mask_x_top][0],
            flash_geom_x[mask_x_top][0]]
    box_y = [flash_geom_y[mask_x_top][0],
             y_secondtolast,
             flash_geom_y[mask_y_top][0],
             flash_geom_y[mask_y_top][-1],
             np.max(flash_geom_y[mask_x_bot]),
             np.min(flash_geom_y[mask_x_bot]),
             flash_geom_y[mask_y_bot][0],
             flash_geom_y[mask_y_bot][-1],
             y_secondtofirst,
             flash_geom_y[mask_x_top][0],
             flash_geom_y[mask_x_top][0]]
#     for i in range(len(box_x)-2):
#         if (box_x[i+1]-box_x[i]) !=0:
#             if (box_x[i+1]-box_x[i])> 0:
#                 x_vals = np.arange(box_x[i], box_x[i+1], 0.001)
#             else:
#                 x_vals = np.arange(box_x[i+1], box_x[i], 0.001)
#             m_box = (box_y[i+1]-box_y[i])/(box_x[i+1]-box_x[i])
#             t_box = box_y[i]- m_box*box_x[i]
#             y_vals = m_box*x_vals + t_box
#             plt.plot(x_vals, y_vals, c = "black")
#             #idx = np.argwhere(np.diff(np.sign(m*x_vals+t - y_vals))).flatten()
# #             if len(idx) !=0:
# #                 intersections.append([x_vals[idx][0], y_vals[idx][0]])
# #             plt.scatter(x_vals[idx], y_vals[idx], facecolors='none', edgecolors="black")
#         else:
#             x_vals = [box_x[i]]
#             if box_y[i]<box_y[i+1]:
#                 y_vals = np.arange(box_y[i], box_y[i+1], 0.001)
#             else:
#                 y_vals = np.arange(box_y[i+1], box_y[i], 0.001)
#             for j in range(len(y_vals)-1):
#                 x_vals.append(x_vals[0])
#             plt.vlines(box_x[i], box_y[i], box_y[i+1], colors ="black")
    return box_x, box_y

In [ ]:
def apply_selection_cut(dict_high_pixel):
    max_counts = 70
    high_pixel_cut = {}
    for nrun in dict_high_pixel.keys():
        high_pixel_cut[nrun] = {}
        values, counts = np.unique(dict_high_pixel[nrun]["pix"], return_counts=True)
        mask_max_counts = counts<max_counts       
        st = set(values[mask_max_counts])
        result = [i for i, e in enumerate(dict_high_pixel[nrun]["pix"]) if e in st]
        high_pixel_cut[nrun]["pix"] = np.array(dict_high_pixel[nrun]["pix"])[result].astype(int)
        high_pixel_cut[nrun]["brightness"] = np.array(dict_high_pixel[nrun]["brightness"])[result]
        high_pixel_cut[nrun]["time"] = np.array(dict_high_pixel[nrun]["time"])[result]-dict_high_pixel[nrun]["time"][0]
    return high_pixel_cut

In [ ]:
@jit
def apply_selection_cut_single_run_jit(high_pixel_single_run):
    max_counts = 70
    #[0,1,2] will correspond to [pixId, time, brightness]
    values, counts = np.unique(high_pixel_single_run[0], return_counts=True)
    high_pixel_cut = 1
    return high_pixel_cut

In [ ]:
print(high_pixel.keys())
for nrun in high_pixel.keys():
    cut_high_pixel = apply_selection_cut_single_run_jit(high_pixel[nrun])
    print(cut_high_pixel.keys())

In [ ]:
a,b = get_box()
print(a)
print(b)
r = []
phi = []
a = np.around(a, 4)
b = np.around(b, 4)
get_lin_fit(track)
for i in range(len(track)):
    plot_from_pix(track[i][0], track[i][1])
x_ranges =[]
for i in range(len(a)-1):
    popt, pcov= curve_fit(lin_fit, [a[i], a[i+1]], [b[i], b[i+1]] ) ##need to ignore bad pcov somehow
    print(popt)
    if a[i]<a[i+1]:
        x_range = np.arange(a[i], a[i+1], 0.001)
    elif a[i]>a[i+1]:
        x_range = np.arange(a[i+1], a[i], 0.001)
    else:
        x_range = np.array([a[i]])
    x_range = np.around(x_range, 4)
    print(min(x_range), max(x_range), len(x_range))
    #plt.plot(x_range, lin_fit(x_range, popt[0], popt[1]))
    x_ranges.append(x_range)
plt.scatter(a,b, c = np.arange(len(a)), cmap = "jet")

.

.

.





Evaluation of data starts here
.

.






.

In [ ]:
%%time
high_pixel = read_data_sets(158000, 175400) #read in data from start to end, maybe remove print 
                                            #statements for better readability
print(list(high_pixel.keys())) #all runs  
nrun = 0
cut_high_pixel = apply_selection_cut(high_pixel) #cuts away all pixels that triggered more than 70 times
for nrun in cut_high_pixel.keys():
    print(nrun)
    track, possible_meteorite = track_sorter(cut_high_pixel[nrun]["pix"],
                                     cut_high_pixel[nrun]["time"],
                                     cut_high_pixel[nrun]["brightness"],
                                    typed_nn_pix)
    #this one is with @jit, so it was done with typed Lists, still needs to be appended to some sort of dictionary or something
    print(nrun, "tracks:", len(track), "meteorite or junk:", len(possible_meteorite))
    if (track[0][0][0] ==-1 and possible_meteorite[0][0][0]==-1):
        print("Run", nrun,"void of (useful)data")
        continue
    if (track[0][0][0]==-1):
        print("Run", nrun,"without satelite tracks")
    else:
        pass
        plot_poss_met(track)   
    if (possible_meteorite[0][0][0]==-1):
        print("Run", nrun,"without meteories")
        continue
    possible_meteorite, trash = adding_possible_meteorites(possible_meteorite)  #Adding together all data that happened at the 
                                                                                #same time, trash is most likely not a meteorite
    counter = 0
    for i in range(len(possible_meteorite)):
        if len(np.unique(possible_meteorite[i][0]))<4:
            continue
        counter+=1
        plot_from_pix(possible_meteorite[i][0], possible_meteorite[i][1])
        plt.show()
    print(counter) # How many meteorites are detected

In [ ]:
nrun = 175118
print(nruns)
track, possible_meteorite = track_sorter(cut_high_pixel[nrun]["pix"],
                                     cut_high_pixel[nrun]["time"],
                                     cut_high_pixel[nrun]["brightness"],
                                    typed_nn_pix)
counter = 0
print(len(possible_meteorite))
for i in range(len(possible_meteorite)):
    if len(np.unique(possible_meteorite[i][0]))<3:
        continue 
    if -1 in possible_meteorite[i][1]:
        continue
    counter+=1
#     plot_from_pix(possible_meteorite[i][0], possible_meteorite[i][1])
#     plt.show()
print(counter)
possible_meteorite, trash = adding_possible_meteorites(possible_meteorite)
print("Adding:")
counter = 0
pixelarray = np.array([])
for i in range(len(possible_meteorite)):
    if len(np.unique(possible_meteorite[i][0]))<4:
        continue
    if -1 in possible_meteorite[i][1]:
        continue
    pixelarray = np.concatenate((pixelarray, np.unique(possible_meteorite[i][0])), axis = None)
print(len(pixelarray))
index_array = np.array([])
for i in range(len(possible_meteorite)):
    if len(np.unique(possible_meteorite[i][0]))<4:
        continue
    if -1 in possible_meteorite[i][1]:
        continue
    
    print("unique pixels:",np.unique(possible_meteorite[i][0]))
    index_to_delete = np.unique(possible_meteorite[i][0])
    for j in range(len(index_to_delete)):
        index_to_delete[j] = len(index_array)+j 
    index_array = np.concatenate((index_array, np.unique(possible_meteorite[i][0])), axis = None) #used just to get the length
    print("index_to_delete:",index_to_delete)
    a = pixelarray
    c = np.delete(a, index_to_delete.astype(int))
    print(a)
    print(c)
    if np.array_equal(np.intersect1d(c,np.unique(possible_meteorite[i][0])),np.unique(possible_meteorite[i][0])):
        continue
    counter+=1
    plot_from_pix(possible_meteorite[i][0], possible_meteorite[i][1])
    plt.show()
print(len(pixelarray))
print(counter)

In [ ]:
@jit
def remove_more_trash(pos_met):
    min_pixel = 3 #minimum pixel number to pass the check
    pixarray = np.array([]) #Concatenate unique pixels of each pos_met track
    index_count = 0
    for i in range(len(pos_met)):
        if (len(np.unique(pos_met[i][0]))<min_pixel or -1 in pos_met[i][1]):
            continue
        pixarray = np.concatenate((pixarray, np.unique(pos_met[i][0])), axis = None)
    for i in range(len(pos_met)):
        if (len(np.unique(pos_met[i][0]))<min_pixel or -1 in pos_met[i][1]):
            continue
        index_to_del = np.unique(pos_met[i][0]) 
        for j in range(len(index_to_del)):
            index_to_del[j] = len(index_count)+j #delete these indices 
        index_count = index_count + len(np.unique(poss_met[i][0]))
    return pos_met_cleaned

In [ ]:

def remove_more_trash_test(pos_met):
    pos_met_clean = pos_met
    
    return pos_met_clean

In [ ]:
%%time 
cut_high_pixel = apply_selection_cut(high_pixel)

In [ ]:
print(type(possible_meteorite[0]))
print(possible_meteorite[0])
a = List(possible_meteorite[0])
print(a)

In [ ]:
nrun = 175118
track, possible_meteorite = track_sorter(cut_high_pixel[nrun]["pix"],
                                     cut_high_pixel[nrun]["time"],
                                     cut_high_pixel[nrun]["brightness"],
                                    typed_nn_pix)
# print(type(possible_meteorite[0]))
# possible_meteorite, trash = adding_possible_meteorites_jit(possible_meteorite)
# print(type(possible_meteorite))
# possible_meteorite_cleaned = remove_more_trash_test(possible_meteorite)

In [ ]:

nrun =158035 
track, possible_meteorite = track_sorter(cut_high_pixel[nrun]["pix"],
                                     cut_high_pixel[nrun]["time"],
                                     cut_high_pixel[nrun]["brightness"],
                                    typed_nn_pix)
print(nrun, "tracks:", len(track), "meteorite or junk:", len(possible_meteorite))
if (track[0][0][0] ==-1 and possible_meteorite[0][0][0]==-1):
    print("Run", nrun,"void of (useful)data")
    pass
if (track[0][0][0]==-1):
    print("Run", nrun,"without satelite tracks")
else:
    plot_poss_met(track)   
if (possible_meteorite[0][0][0]==-1):
    print("Run", nrun,"without meteories")
    pass
possible_meteorite, trash = adding_possible_meteorites(possible_meteorite)
print(len(possible_meteorite), len(trash))
counter = 0
for i in range(len(possible_meteorite)):
    if len(np.unique(possible_meteorite[i][0]))<3:
        continue
#     print(np.unique(possible_meteorite[i][0]))
    counter+=1
    plot_from_pix(possible_meteorite[i][0], possible_meteorite[i][1])
    plt.show()
print(counter)

In [ ]:
nrun =158035        
print(len(np.unique(high_pixel[nrun]["pix"])))
i=0
values, counts = np.unique(high_pixel[nrun]["pix"], return_counts=True)
print(len(values))
while len(values)>200:
    mask = high_pixel[nrun]["brightness"]>900+i*50
    print("Threshold",900+i*50)
    print(len(values))
    values, counts = np.unique(high_pixel[nrun]["pix"][mask], return_counts=True)
    i+=1
print(len(high_pixel[nrun]["time"][mask]))
mask_max_counts = counts<max_counts
print("new treshold at", 900+i*50)
plot_from_pix(high_pixel[nrun]["pix"][mask][:1000000], high_pixel[nrun]["time"][mask][:1000000])
plt.show()

In [ ]:
%%time
# Remove all pixels that have more than max_counts occurences, since those are most of the time no satellite trails.
# losses along a  trail have to be accepted. Max_counts is deliberately chosen generously
# if unique pixels too many make new threshold
#################Improved version without x/y pos#################
#################Use this one#################
high_pixel = test
telId = 5
max_counts = 50
for nrun in nruns:
    trails_high_avg[nrun] = {}
    trails_high_avg[nrun][telId] = {}
    try:
        values, counts = np.unique(high_pixel[nrun]["pix"+str(telId)], return_counts=True)
        
        mask_max_counts = counts<max_counts
        i=0
        while len(values)>400:
            mask = high_pixel[nrun][telId]>900+i*50
            values, counts = np.unique(high_pixel[nrun]["pix"+str(telId)][mask], return_counts=True)
            i+=1
        mask_max_counts = counts<max_counts
        print("new treshold at", 900+i*50)        
        st = set(values[mask_max_counts])
        result = [i for i, e in enumerate(high_pixel[nrun]["pix"+str(telId)]) if e in st]
        trails_high_avg[nrun][telId]["z"] = np.array(high_pixel[nrun]["tmp"+str(telId)][result]-high_pixel[nrun]["tmp"+str(telId)][0])
        trails_high_avg[nrun][telId]["pix ID"] = np.array(high_pixel[nrun]["pix"+str(telId)][result])
        trails_high_avg[nrun][telId]["pix ID"] = trails_high_avg[nrun][telId]["pix ID"].astype(int)
        trails_high_avg[nrun][telId]["brightness"] = np.array(high_pixel[nrun][telId][result])
        plot_from_pix(trails_high_avg[nrun][telId]["pix ID"], trails_high_avg[nrun][telId]["z"])
        draw_box()
        plt.show()
    except:
        print("No CT 5 for run ", nrun )
        pass

In [ ]:
%%time
########################################################################################
############Good display for tracks and possible meteories##############################
########################################################################################
########################################################################################
#########################DO NOT DELETE##################################################
########################################################################################
for nrun in nruns:
    print(nrun)
    track, possible_meteorite = track_sorter(trails_high_avg[nrun][telId]["pix ID"],
                                     trails_high_avg[nrun][telId]["z"],
                                     trails_high_avg[nrun][telId]["brightness"],
                                    typed_nn_pix)
    if (track[0][0][0] ==-1 and possible_meteorite[0][0][0]==-1):
        print("Run", nrun,"void of (useful)data")
        continue
    if (track[0][0][0]==-1):
        print("Run", nrun,"without satelite tracks")
    else:
        plot_poss_met(track)   
    if (possible_meteorite[0][0][0]==-1):
        print("Run", nrun,"without meteories")
        continue
    possible_meteorite, trash = adding_possible_meteorites(possible_meteorite)
    plot_poss_met(possible_meteorite)
    

In [ ]:
%%time 
for nrun in nruns:
    try:
        a,b = get_neighbour_pixel_entries(np.array(trails_high_avg[nrun][telId]["x"]),
                                          np.array(trails_high_avg[nrun][telId]["y"]),
                                          np.array(trails_high_avg[nrun][telId]["z"]),
                                          np.array(trails_high_avg[nrun][telId]["pix ID"]),
                                          np.array(trails_high_avg[nrun][telId]["brightness"]))
    except:
        pass

In [ ]:
%%time 
### Sort data into tracks and meteories/junk
#Adjust get_neighbour_pixel_entries to be faster in evaluation
telId = 5
trackruns = {}
possible_meteorites = {}
for nrun in nruns:
    trackruns[nrun] = {}
    possible_meteorites[nrun] = {}
    try:
        timeout = 10   # [seconds]

        timeout_start = time.time()
        while time.time() < timeout_start + timeout:
            trackruns[nrun], possible_meteorites[nrun] = get_neighbour_pixel_entries(np.array(trails_high_avg[nrun][telId]["x"]),
                                    np.array(trails_high_avg[nrun][telId]["y"]),
                                    np.array(trails_high_avg[nrun][telId]["z"]),
                                    np.array(trails_high_avg[nrun][telId]["pix ID"]),
                                    np.array(trails_high_avg[nrun][telId]["brightness"]) )
            break
    except:
        print(nrun, "ERROR")
        pass

In [ ]:
%%time
#timeout testing
timeout = 30   # [seconds]

timeout_start = time.time()
i=0
while time.time() < timeout_start + timeout:
    print(i)
    i+=1

In [ ]:
%%time
#Adding together all the meteorite tracks of the same/close time:
for nrun in nruns: 
    temp = list(possible_meteorites[nrun])
    for N in reversed(list(possible_meteorites[nrun].keys())):
        #print(possible_meteorites[nrun][np.array(list(possible_meteorites[nrun].keys()))[0]].keys())
        try:
            try:
                try:
                    res = temp[temp.index(N) - 1]
                except (ValueError, IndexError):
                    res = None
                #Adding:
                if (len(set(possible_meteorites[nrun][N]["z"]) & set(possible_meteorites[nrun][res]["z"]))>0 or 
                   len(set(possible_meteorites[nrun][N]["z"]-1) & set(possible_meteorites[nrun][res]["z"]))>0):
                    possible_meteorites[nrun][res]["x"] =  possible_meteorites[nrun][res]["x"]+ possible_meteorites[nrun][N]["x"]
                    possible_meteorites[nrun][res]["y"] =  possible_meteorites[nrun][res]["y"]+ possible_meteorites[nrun][N]["y"]
                    possible_meteorites[nrun][res]["z"] =  possible_meteorites[nrun][res]["z"]+ possible_meteorites[nrun][N]["z"]
                    possible_meteorites[nrun][res]["pix"] =  possible_meteorites[nrun][res]["pix"]+ possible_meteorites[nrun][N]["pix"]
                    possible_meteorites[nrun][res]["brightness"] =  possible_meteorites[nrun][res]["brightness"] + possible_meteorites[nrun][N]["brightness"]
                    possible_meteorites[nrun].pop(N, None)
                    continue
            except:
                if len(possible_meteorites[nrun][N].keys())==0:
                    possible_meteorites[nrun].pop(N, None)
                pass
            #Delete, if remaining data is no track, might have to be extended/adjusted by linear fit
            if len(set(possible_meteorites[nrun][N]["pix"]))<3:
                possible_meteorites[nrun].pop(N, None)
        except:
            pass

In [ ]:
%%time
#only for testing, can be deleted if neccessary, just for display
for nrun in nruns:
    for N in possible_meteorites[nrun]:
        try:
            print(nrun, N, set(possible_meteorites[nrun][N]["z"]))
            if len(set(possible_meteorites[nrun][N]["pix"]))<3:
                continue
            if len(set(possible_meteorites[nrun][N]["z"]))>2:
                plot_scatter(possible_meteorites[nrun][N]["x"], 
                     possible_meteorites[nrun][N]["y"],
                     possible_meteorites[nrun][N]["brightness"])
                draw_box(0,0)
                plt.show()
            
            plot_scatter(possible_meteorites[nrun][N]["x"], 
                     possible_meteorites[nrun][N]["y"],
                     possible_meteorites[nrun][N]["z"])
            draw_box(0,0)
            plt.show()
        except:
            print(N, possible_meteorites[nrun][N].keys())

In [ ]:
%%time
#just for displaying tracks, not optimal, but gist is visible
for nrun in nruns:
    cax = {}
    counter = 0
    counter_max_time = 0
    if (len(trackruns[nrun])==0 and len(possible_meteorites[nrun])==0):
        continue
    fig, ax = plt.subplots()
    for N in possible_meteorites[nrun]:
        counter+=1
        if np.max(possible_meteorites[nrun][N]["z"]-possible_meteorites[nrun][N]["z"][0])>= counter_max_time:
            counter_index = counter
            counter_max_time = np.max(possible_meteorites[nrun][N]["z"]-possible_meteorites[nrun][N]["z"][0])
        cax[counter] = ax.scatter(possible_meteorites[nrun][N]["x"], 
                     possible_meteorites[nrun][N]["y"],
                     c = possible_meteorites[nrun][N]["z"]- possible_meteorites[nrun][N]["z"][0], cmap = "jet")
    for N in trackruns[nrun]:
        counter+=1
        if np.max(trackruns[nrun][N]["z"]-trackruns[nrun][N]["z"][0])>= counter_max_time:
            counter_index = counter
            counter_max_time = np.max(trackruns[nrun][N]["z"]-trackruns[nrun][N]["z"][0])
        cax[counter] = ax.scatter(trackruns[nrun][N]["x"], 
        trackruns[nrun][N]["y"],
        c = trackruns[nrun][N]["z"]- trackruns[nrun][N]["z"][0], cmap = "jet")
    draw_box(0,0)
    ax.set_xlim(-1.25,1.25)
    ax.set_ylim(-1.25,1.25)
    ax.set_xlabel("Camera x-pos")
    ax.set_ylabel("Camera y-pos")
    ax.set_title("Run {}".format(nrun)+", CT {}".format(telId))
    cbar = fig.colorbar(cax[counter_index])
    cbar.set_label("Time in track [s]")
    plt.show()

In [ ]:
%%time
satellite_data = {}
#sort into statistics files
for nrun in nruns:
    satellite_data[nrun] = {}
    i = 0
    if len(trackruns[nrun]) == 0:
        satellite_data[nrun] = "No data"
        continue
    for N in trackruns[nrun]:
        if len(trackruns[nrun][N].keys())==0:
            continue
        i+=1
        satellite_data[nrun][i] = {}
        satellite_data[nrun][i]["time in run"] = trackruns[nrun][N]["z"][0]
        satellite_data[nrun][i]["duration"] = np.max(trackruns[nrun][N]["z"]-trackruns[nrun][N]["z"][0])
        satellite_data[nrun][i]["mean brightness"] = int(np.average(trackruns[nrun][N]["brightness"]))
        satellite_data[nrun][i]["mean brightness std"] = int(np.std(trackruns[nrun][N]["brightness"]))
        satellite_data[nrun][i]["N pix"] = len(set(trackruns[nrun][N]["pix"]))
        width_nrun, std_width_nrun = get_track_width(trackruns[nrun][N]["x"], trackruns[nrun][N]["y"])
        satellite_data[nrun][i]["mean width"] = width_nrun
        satellite_data[nrun][i]["std width"] = std_width_nrun
        popt, pcov = curve_fit(lin_fit, trackruns[nrun][N]["x"], trackruns[nrun][N]["y"])
        satellite_data[nrun][i]["linear fit parameter"] = (round(popt[0], 5), round(popt[1],5))
        

In [ ]:
print(satellite_data)

In [ ]:
%%time
mean_brightness = []
mean_width = []
new_nruns = []
directions = []
durations = []
for nrun in nruns:
    if satellite_data[nrun]=="No data":
        #print(satellite_data[nrun],"for", nrun)
        continue
    for i in satellite_data[nrun]:
        mean_brightness.append(satellite_data[nrun][i]["mean brightness"])
        mean_width.append(satellite_data[nrun][i]["mean width"])
        new_nruns.append(nrun)
        directions.append(satellite_data[nrun][i]["linear fit parameter"][0])
        durations.append(satellite_data[nrun][i]["duration"])
print(np.average(mean_brightness), np.std(mean_brightness))
plt.hist(mean_brightness)
plt.xlabel("mean brightness [MHz]")
plt.ylabel("number")
plt.show()
plt.hist(directions)
plt.show()
plt.hist(durations)
plt.show()
mask = np.array(new_nruns) < 160000
plt.scatter(np.array(mean_brightness)[mask],np.array(mean_width)[mask], c =np.array(new_nruns)[mask], cmap="jet")
plt.xlabel("mean brightness [MHz]")
plt.ylabel("mean width [pix]")
cbar = plt.colorbar()
cbar.set_label("nrun")
plt.show()

In [ ]:
%%time
nrun = 174949
telId = 5
#Display satellites and meteories all in one plot
fig, ax = plt.subplots(figsize=(8,6))
draw_box()
fig2, ax2 = plt.subplots(figsize=(8,6))
draw_box()
fig3, ax3 = plt.subplots(figsize=(8,6))
draw_box()
thresh = 3500
print(possible_meteorites[nrun].keys())
for N in range(np.max(list(possible_meteorites[nrun].keys()))+1):
    if N not in possible_meteorites[nrun]:
        continue
    if len(possible_meteorites[nrun][N].keys()) == 0:
        continue
    if len(set(possible_meteorites[nrun][N]["pix"]))< 6:
        possible_meteorites[nrun].pop(N, None)
        continue
#     print(set(possible_meteorites[nrun][N]["pix"]))
    ax.scatter(possible_meteorites[nrun][N]["x"],
                possible_meteorites[nrun][N]["y"],
                c = possible_meteorites[nrun][N]["z"]-possible_meteorites[nrun][N]["z"][0],  cmap = "jet")
    pix = list(possible_meteorites[nrun][N]["pix"])
    z = list(possible_meteorites[nrun][N]["z"]-possible_meteorites[nrun][N]["z"][0])
    brightness = list(possible_meteorites[nrun][N]["brightness"])
    #print(set(possible_meteorites[nrun][N]["z"]- possible_meteorites[nrun][N]["z"][0]))
    b = np.where(np.array(possible_meteorites[nrun][N]["brightness"]) > thresh, thresh, np.array(possible_meteorites[nrun][N]["brightness"]))
    ax2.scatter(possible_meteorites[nrun][N]["x"],
                possible_meteorites[nrun][N]["y"],
                c = b, cmap = "jet")
    
    popt, pcov = curve_fit(lin_fit, possible_meteorites[nrun][N]["x"],possible_meteorites[nrun][N]["y"])
    ax.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt), "--", c = "black")
    ax2.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt), "--", c = "black")
    ax3.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt), "--", c = "black")
#print(trackruns[nrun].keys())
for N in trackruns[nrun].keys():
    cax = ax.scatter(trackruns[nrun][N]["x"],
                trackruns[nrun][N]["y"],
                c = trackruns[nrun][N]["z"]- trackruns[nrun][N]["z"][0], cmap = "jet")
    pix = pix + list(trackruns[nrun][N]["pix"])
    z_pre = np.array(trackruns[nrun][N]["z"])-trackruns[nrun][N]["z"][0]
    z = z + list(trackruns[nrun][N]["z"]-trackruns[nrun][N]["z"][0])
    brightness = brightness + list(trackruns[nrun][N]["brightness"])
    #print(set(trackruns[nrun][N]["z"]- trackruns[nrun][N]["z"][0]))
    popt, pcov = curve_fit(lin_fit, trackruns[nrun][N]["x"],trackruns[nrun][N]["y"])
    
    b = np.where(np.array(trackruns[nrun][N]["brightness"]) > thresh, thresh, np.array(trackruns[nrun][N]["brightness"]))
    
    
    cax2 = ax2.scatter(trackruns[nrun][N]["x"],
                trackruns[nrun][N]["y"],
                c = b, cmap = "jet")
    popt, pcov = curve_fit(lin_fit, trackruns[nrun][N]["x"],trackruns[nrun][N]["y"])
    ax.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt),  "--", c = "black")
    ax2.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt),  "--", c = "black")
    ax3.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt),  "--", c = "black")
    
ax.set_xlim(-1.25,1.25)
ax.set_ylim(-1.25,1.25)
ax2.set_xlim(-1.25,1.25)
ax2.set_ylim(-1.25,1.25)
draw_box()
cbar = fig.colorbar(cax)
#cbar.ax.set_yticklabels(['first in track', 'last in track '])
ax.set_title("Timing during each track")
ax.set_xlabel("Camera x-pos")
ax.set_ylabel("Camera y-pos")
cbar2 = fig2.colorbar(cax2) #ticks = [1000,2000,3000,4000, 6000])
cbar2.set_label("MHz")
#cbar2.ax2.set_yticklabels(['first in track', 'last in track '])
ax2.set_title("Brightness")
ax2.set_xlabel("Camera x-pos")
ax2.set_ylabel("Camera y-pos")
for i in range(len(pix)):
    pix[i] = int(pix[i])

ax3.set_xlim(-1.25,1.25)
ax3.set_ylim(-1.25,1.25)
cax3 = ax3.scatter(np.array(geom_hess5_xc_from_root)[np.array(pix)], 
            np.array(geom_hess5_yc_from_root)[np.array(pix)],
            c = z, cmap ="rainbow")
cbar3 = fig3.colorbar(cax3)
ax3.set_title("Timing during each track")
ax3.set_xlabel("Camera x-pos")
ax3.set_ylabel("Camera y-pos")
cbar3.set_label("Time in track [s]")

plt.show()

In [ ]:
# print(possible_meteorites[nrun].keys())
fig, ax = plt.subplots(figsize=(8,6))
draw_box()
fig2, ax2 = plt.subplots(figsize=(8,6))
draw_box()
fig3, ax3 = plt.subplots(figsize=(8,6))
draw_box()
thresh = 2500
for N in range(np.max(list(possible_meteorites[nrun].keys()))+1):
    if N not in possible_meteorites[nrun]:
        continue
    if len(set(possible_meteorites[nrun][N]["pix"]))< 6:
        possible_meteorites[nrun].pop(N, None)
        continue
    print(set(possible_meteorites[nrun][N]["pix"]))
    ax.scatter(possible_meteorites[nrun][N]["x"],
                possible_meteorites[nrun][N]["y"],
                c = possible_meteorites[nrun][N]["z"]-possible_meteorites[nrun][N]["z"][0],  cmap = "jet")
    pix = list(possible_meteorites[nrun][N]["pix"])
    z = list(possible_meteorites[nrun][N]["z"]-possible_meteorites[nrun][N]["z"][0])
    brightness = list(possible_meteorites[nrun][N]["brightness"])
    print(set(possible_meteorites[nrun][N]["z"]- possible_meteorites[nrun][N]["z"][0]))
    b = np.where(np.array(possible_meteorites[nrun][N]["brightness"]) > thresh, thresh, np.array(possible_meteorites[nrun][N]["brightness"]))
    ax2.scatter(possible_meteorites[nrun][N]["x"],
                possible_meteorites[nrun][N]["y"],
                c = b, cmap = "jet")
    
    popt, pcov = curve_fit(lin_fit, possible_meteorites[nrun][N]["x"],possible_meteorites[nrun][N]["y"])
    ax.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt), "--", c = "black")
    ax2.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt), "--", c = "black")
    ax3.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt), "--", c = "black")
print(trackruns[nrun].keys())
for N in trackruns[nrun].keys():
    cax = ax.scatter(trackruns[nrun][N]["x"],
                trackruns[nrun][N]["y"],
                c = trackruns[nrun][N]["z"]- trackruns[nrun][N]["z"][0], cmap = "jet")
    pix = pix + list(trackruns[nrun][N]["pix"])
    z_pre = np.array(trackruns[nrun][N]["z"])-trackruns[nrun][N]["z"][0]
    z = z + list(trackruns[nrun][N]["z"]-trackruns[nrun][N]["z"][0])
    brightness = brightness + list(trackruns[nrun][N]["brightness"])
    print(set(trackruns[nrun][N]["z"]- trackruns[nrun][N]["z"][0]))
    popt, pcov = curve_fit(lin_fit, trackruns[nrun][N]["x"],trackruns[nrun][N]["y"])
    
    b = np.where(np.array(trackruns[nrun][N]["brightness"]) > thresh, thresh, np.array(trackruns[nrun][N]["brightness"]))
    
    
    cax2 = ax2.scatter(trackruns[nrun][N]["x"],
                trackruns[nrun][N]["y"],
                c = b, cmap = "jet")
    popt, pcov = curve_fit(lin_fit, trackruns[nrun][N]["x"],trackruns[nrun][N]["y"])
    ax.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt),  "--", c = "black")
    ax2.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt),  "--", c = "black")
    ax3.plot(np.arange(-1.,1.01), lin_fit(np.arange(-1.,1.01), *popt),  "--", c = "black")
    
ax.set_xlim(-1.25,1.25)
ax.set_ylim(-1.25,1.25)
ax2.set_xlim(-1.25,1.25)
ax2.set_ylim(-1.25,1.25)
draw_box()
cbar = fig.colorbar(cax)
#cbar.ax.set_yticklabels(['first in track', 'last in track '])
ax.set_title("Timing during each track")
ax.set_xlabel("Camera x-pos")
ax.set_ylabel("Camera y-pos")
cbar2 = fig2.colorbar(cax2) #ticks = [1000,2000,3000,4000, 6000])
cbar2.set_label("MHz")
#cbar2.ax2.set_yticklabels(['first in track', 'last in track '])
ax2.set_title("Brightness")
ax2.set_xlabel("Camera x-pos")
ax2.set_ylabel("Camera y-pos")
fig.savefig("D:\\Masterarbeit ECAP\\Run-174949-all-tracks-timing.jpg")
fig2.savefig("D:\\Masterarbeit ECAP\\Run-174949-all-tracks-brightness.jpg")
for i in range(len(pix)):
    pix[i] = int(pix[i])

ax3.set_xlim(-1.25,1.25)
ax3.set_ylim(-1.25,1.25)
cax3 = ax3.scatter(np.array(geom_hess5_xc_from_root)[np.array(pix)], 
            np.array(geom_hess5_yc_from_root)[np.array(pix)],
            c = z, cmap ="rainbow")
cbar3 = fig3.colorbar(cax3)
ax3.set_title("Timing during each track")
ax3.set_xlabel("Camera x-pos")
ax3.set_ylabel("Camera y-pos")
cbar3.set_label("Time in track [s]")

fig3.savefig("D:\\Masterarbeit ECAP\\Run-174949-all-tracks-timing-better.jpg")
plt.show()

In [ ]:
%%time 
nrun = 174949
telId=5

fig1, ax1 = plt.subplots(figsize=(8,6))
draw_box()
fig2, ax2 = plt.subplots(figsize=(8,6))
draw_box()
thresh = 2650

print(len(high_pixel[nrun]["x-pos"+str(telId)]))
mask = high_pixel[nrun][telId]<6100
b =  np.where(np.array(high_pixel[nrun][telId])>thresh, thresh, np.array(high_pixel[nrun][telId]))
cax2 = ax2.scatter(high_pixel[nrun]["x-pos"+str(telId)], 
                 high_pixel[nrun]["y-pos"+str(telId)], 
                 c = b, cmap = "jet")
cax1 = ax1.scatter(high_pixel[nrun]["x-pos"+str(telId)], 
                 high_pixel[nrun]["y-pos"+str(telId)], 
                 c = high_pixel[nrun]["tmp"+str(telId)] -high_pixel[nrun]["tmp"+str(telId)][0], cmap = "jet")


ax1.set_title("Timing in s")
ax1.set_xlabel("Camera x-pos")
ax1.set_ylabel("Camera y-pos")
cbar1 = fig1.colorbar(cax1)
ax2.set_title("Brightness")
ax2.set_xlabel("Camera x-pos")
ax2.set_ylabel("Camera y-pos")
cbar2 = fig2.colorbar(cax2)
fig1.savefig("D:\\Masterarbeit ECAP\\Run-174949-uncleaned-timing.jpg")
fig2.savefig("D:\\Masterarbeit ECAP\\Run-174949-uncleaned-brightness.jpg")
plt.show()

In [ ]:
%%time

nrun = 174949
telId = 5
# x = np.array(trails_high_avg[nrun][telId]["x"])
# y = np.array(trails_high_avg[nrun][telId]["y"])
# z = np.array(trails_high_avg[nrun][telId]["z"])
# pix = np.array(trails_high_avg[nrun][telId]["pix ID"])
# brightness = np.array(trails_high_avg[nrun][telId]["brightness"])

x = np.array(high_pixel[nrun]["x-pos"+str(telId)])
y = np.array(high_pixel[nrun]["y-pos"+str(telId)])
z = np.array(high_pixel[nrun]["tmp"+str(telId)]- high_pixel[nrun]["tmp"+str(telId)][0])
pix = np.array(high_pixel[nrun]["pix"+str(telId)])
brightness = np.array(high_pixel[nrun][telId])
Pix = [ (pix[i],i) for i in np.arange(len(pix))]
Pix.sort()
#Probiere nochmal mit List comprehension?
sorted_pix,permutation = zip(*Pix)
sorted_pix = np.array(sorted_pix)
permutation = np.array(permutation)
#print(len(set(high_pixel[nrun]["pix"+str(telId)])))
# for pix in set(high_pixel[nrun]["pix"+str(telId)]):
#     print(int(pix), end = ", ")
mean_pix_pix = []
mean_pix_x = []
mean_pix_y = []
mean_pix_z = []
mean_pix_brightness = []
counter = 0
print(len(set(pix)))
for i in range(len(x)):
    if i == 0:
        #mean_x = [x[permutation][i]]
        #mean_y = [y[permutation][i]]
        mean_z = [z[permutation][i]]
        mean_pix = [pix[permutation][i]]
        mean_brightness = [brightness[permutation][i]]
        continue
    if pix[permutation][i] == pix[permutation][i-1]:
        
        #mean_x.append(x[permutation][i])
        #mean_y.append(y[permutation][i])
        mean_z.append(z[permutation][i])
        mean_pix.append(pix[permutation][i])
        mean_brightness.append(brightness[permutation][i])
    else:
        counter +=1
        print(counter)
        #mean_pix_x.append(np.average(mean_x))
        #mean_pix_y.append(np.average(mean_y))
        mean_pix_z.append(np.average(mean_z))
        mean_pix_pix.append(np.average(mean_pix))
        mean_pix_brightness.append(np.average(mean_brightness))
        
        #mean_x = [x[permutation][i]]
        #mean_y = [y[permutation][i]]
        mean_z = [z[permutation][i]]
        mean_pix = [pix[permutation][i]]
        mean_brightness = [brightness[permutation][i]]
fig, ax = plt.subplots(figsize =(8,6))    
draw_box()

cax = ax.scatter(np.array(mean_pix_x),
                 np.array(mean_pix_y),
                 c = np.array(mean_pix_brightness), cmap = "jet" )
fig.colorbar(cax)
plt.show()

In [ ]:
print(mean_pix_pix)
for i in range(len(mean_pix_pix)):
    mean_pix_pix[i] = int(mean_pix_pix[i])
thresh = 4100
fig, ax = plt.subplots(figsize =(8,6))
b = np.where(np.array(mean_pix_brightness) > thresh, thresh, np.array(mean_pix_brightness))
    
cax =  ax.scatter(np.array(geom_hess5_xc_from_root)[np.array(mean_pix_pix)], 
           np.array(geom_hess5_yc_from_root)[np.array(mean_pix_pix)], 
           c = b, cmap = "jet")
draw_box()
ax.set_xlim(-1.25,1.25)
ax.set_ylim(-1.25,1.25)
ax.set_title("Average brightness in MHz")
ax.set_xlabel("Camera x-pos")
ax.set_ylabel("Camera y-pos")
cbar = fig.colorbar(cax)
fig.savefig("D:\\Masterarbeit ECAP\\First Plots\\run174949-average-brightness.jpg")
plt.show()

In [ ]:
nrun = 174949
print(possible_meteorites[nrun].keys())
for N in reversed(list(possible_meteorites[nrun].keys())):
    #print(possible_meteorites[nrun][np.array(list(possible_meteorites[nrun].keys()))[0]].keys())
    try:
        try:
            if len(set(possible_meteorites[nrun][N]["z"]) & set(possible_meteorites[nrun][N-1]["z"]))>0:
                possible_meteorites[nrun][N-1]["x"] =  possible_meteorites[nrun][N-1]["x"]+ possible_meteorites[nrun][N]["x"]
                possible_meteorites[nrun][N-1]["y"] =  possible_meteorites[nrun][N-1]["y"]+ possible_meteorites[nrun][N]["y"]
                possible_meteorites[nrun][N-1]["z"] =  possible_meteorites[nrun][N-1]["z"]+ possible_meteorites[nrun][N]["z"]
                possible_meteorites[nrun][N-1]["pix"] =  possible_meteorites[nrun][N-1]["pix"]+ possible_meteorites[nrun][N]["pix"]
                possible_meteorites[nrun][N-1]["brightness"] =  possible_meteorites[nrun][N-1]["brightness"] + possible_meteorites[nrun][N]["brightness"]
                possible_meteorites[nrun].pop(N, None)
                continue
        except:
            pass
        print(set(possible_meteorites[nrun][N]["z"]))
        popt, pcov = curve_fit(lin_fit,possible_meteorites[nrun][N]["x"],possible_meteorites[nrun][N]["y"])
        draw_box()
        plot_scatter(possible_meteorites[nrun][N]["x"],
                    possible_meteorites[nrun][N]["y"],
                    possible_meteorites[nrun][N]["z"])
        plt.plot(np.arange(-1.25,1.25, 0.01), lin_fit(np.arange(-1.25,1.25, 0.01), *popt))
        try:
            print("Saving...")
            plt.savefig("D:\\Masterarbeit ECAP\\Run-174949-meteorite.jpg")
        except:
            print("saving did not work, try closing the file")
        plt.show()
        plot_scatter(possible_meteorites[nrun][N]["x"],
                    possible_meteorites[nrun][N]["y"],
                    possible_meteorites[nrun][N]["brightness"])
        draw_box()
        plt.show()
    except:
        try:
            print("error", set(possible_meteorites[nrun][N]["z"]))
            pass
        except:
            possible_meteorites[nrun].pop(N, None)
            print("no time found")
            
print(trackruns[nrun].keys())
counter = 0
for N in (list(trackruns[nrun].keys())):
    if N not in trackruns[nrun].keys():
        continue
    if "x" not in trackruns[nrun][N].keys():
        trackruns[nrun].pop(N, None)
        continue
    if len(set(trackruns[nrun][N]["pix"]))>4:
        popt, pcov = curve_fit(lin_fit,trackruns[nrun][N]["x"],trackruns[nrun][N]["y"])
        print(popt)
        draw_box()
        plot_scatter(trackruns[nrun][N]["x"],
                    trackruns[nrun][N]["y"],
                    trackruns[nrun][N]["z"])

        plt.plot(np.arange(-1.25,1.25, 0.01), lin_fit(np.arange(-1.25,1.25, 0.01), *popt))
        counter+=1
        plt.savefig("D:\\Masterarbeit ECAP\\Run-174949-track "+str(counter)+".jpg")
        plt.show()
        mask = np.array(trackruns[nrun][N]["brightness"])<np.average(np.array(trackruns[nrun][N]["brightness"]))+0.1*np.std(np.array(trackruns[nrun][N]["brightness"]))
        mask = np.array(trackruns[nrun][N]["brightness"])>np.average(np.array(trackruns[nrun][N]["brightness"]))-0.1*np.std(np.array(trackruns[nrun][N]["brightness"]))
        mask = np.array(trackruns[nrun][N]["brightness"]) > 0
        plot_scatter(np.array(trackruns[nrun][N]["x"])[mask],
                    np.array(trackruns[nrun][N]["y"])[mask],
                    np.array(trackruns[nrun][N]["brightness"])[mask])
        draw_box()
        plt.show()

    else:
        trackruns[nrun].pop(N, None)
   

In [ ]:
%%time
telId = 5
nrun = 174949
plot_scatter(-high_pixel[nrun]["y-pos"+str(telId)], 
             -high_pixel[nrun]["x-pos"+str(telId)], 
             high_pixel[nrun]["tmp"+str(telId)])
plt.show()

In [ ]:
for nrun in nruns:
    print(nrun)
    try:
        for N in range(1,np.max(list(possible_meteorites[nrun].keys()))+1):
            try:
                if len(np.unique(possible_meteorites[nrun][N]["pix"]))>0:
                    print(N, np.unique(possible_meteorites[nrun][N]["z"]))
                    popt, pcov = curve_fit(lin_fit, possible_meteorites[nrun][N]["x"], possible_meteorites[nrun][N]["y"])
                    
                    intersections = draw_box()
                    print("values for m, t:", np.around(popt, 6))
                    print("Covariances",np.around(np.diag(pcov),6))
                    
                    plt.plot(np.arange(-1.25,1.25, 0.01), lin_fit(np.arange(-1.25,1.25, 0.01), *popt))
                    plot_scatter(possible_meteorites[nrun][N]["x"], 
                                 possible_meteorites[nrun][N]["y"], 
                                 possible_meteorites[nrun][N]["z"])
                    plt.show()
            except:
                print("ERROR")
                pass
    except:
        pass

In [ ]:
#appending new_nruns to nruns and delete all empty files
os.chdir("D:\\Masterarbeit ECAP\\First plots\\eval_data\\high_pixel")
first_run = 175000
runs_directory = "run"+str(first_run)+"-"+str(first_run+199)
if (first_run % 200==0):
    pass
else:
    print("First run not divisible by 200")
    sys.exit()
os.chdir(runs_directory)
files = os.listdir(os.curdir)
print(files)
nruns = list(nruns)
new_nruns = []
for i in range(len(files)):
    try:
        new_nruns.append(int(files[i]))
        nruns.append(int(files[i]))
    except:
        print("No runs in folder/not a folder:", files[i])
nruns = sorted(np.unique(nruns))
print(new_nruns)
print(nruns)
os.chdir("D:\\Masterarbeit ECAP\\First plots\\eval_data")
print(os.getcwd())
for nrun in new_nruns:
    high_pixel[nrun] = {}
    high_path = "high_pixel\\"+runs_directory+"\\"+str(nrun)+"\\"
    path[nrun] = {}
    path[nrun]["high"] = "high_pixel\\"+runs_directory+"\\"+str(nrun)+"\\"
    for key in path[nrun]:
        try:
            file_path = path[nrun][key]
            for i in range(len(file_path)):
                file_name = os.listdir(path[nrun][key])[i]
                if os.path.getsize(file_path+file_name) == 0:
                    print("Empty file", file_path+file_name ,"is being removed")
                    os.remove(file_path+file_name)
        except:
            pass

In [ ]:
%%time
for nrun in nruns:
    telId = 1
    for telId in range(1,5):
        fig= plt.figure(figsize=(6.5,5), )
        try:
            print("CT", telId, "of run", nrun, end =": ")
            plt.scatter(geom_hess1_xc_from_root, geom_hess1_yc_from_root, c=avg_pix[nrun][telId], s=70, cmap='turbo')
            plt.colorbar()
            plt.show()
            print
        except:
            print("No Data")
            plt.close()
    fig= plt.figure(figsize=(10,8), )
    try:
        
        telId = 5
        print("CT", telId, "of run", nrun, end =": ")
        plt.scatter(geom_hess5_xc_from_root, geom_hess5_yc_from_root, c=avg_pix[nrun][telId], s=30, cmap='rainbow')
        plt.colorbar()
        plt.show()
    except: 
        print("No Data")
        plt.close()
        

In [ ]:
possible_peak_times = {}
for nrun in nruns:
    print("Run number ",nrun)
    telId = 1
    #test = avg_tmp[nrun]["tmp"+str(telId)]
    try:
        print(avg_tmp[nrun]["tmp"+str(telId)][0])
    except:
        pass
    fig = plt.figure(figsize=(12,6), )
   
    possible_peak_times[nrun]={}
    
    for telId in range(1,6):
    #    print("1st entry: " ,avg_tmp[nrun]["tmp"+str(telId)][0])
        try:
            possible_peak_times[nrun][telId] = {}
            tmp0 = avg_tmp[nrun]["tmp"+str(telId)][0]
            for i in range(len(avg_tmp[nrun]["tmp"+str(telId)])):            
                avg_tmp[nrun]["tmp"+str(telId)][i] = avg_tmp[nrun]["tmp"+str(telId)][i]- tmp0

                if telId==5:
                    if avg_tmp[nrun][telId][i]>np.average(avg_tmp[nrun][telId][i-10:i+10])+1:
                        possible_peak_times[nrun][telId][round(avg_tmp[nrun]["tmp"+str(telId)][i],1)] =  avg_tmp[nrun][telId][i]
                        #print("CT", telId, ": ", round(avg_tmp[nrun]["tmp"+str(telId)][i],1), ", " , avg_tmp[nrun][telId][i])
                else:
                    if avg_tmp[nrun][telId][i]>np.average(avg_tmp[nrun][telId][i-3:i+3])+4:
                        possible_peak_times[nrun][telId][round(avg_tmp[nrun]["tmp"+str(telId)][i],1)] =  avg_tmp[nrun][telId][i]
                        #print("CT", telId, ": ", round(avg_tmp[nrun]["tmp"+str(telId)][i],1), ", " , avg_tmp[nrun][telId][i])

            print("number of unique times vs time diff: ", len(np.unique(avg_tmp[nrun]["tmp"+str(telId)])), "vs", 
                  round(np.max(avg_tmp[nrun]["tmp"+str(telId)])-np.min(avg_tmp[nrun]["tmp"+str(telId)]),1))
            plt.scatter(avg_tmp[nrun]["tmp"+str(telId)], avg_tmp[nrun][telId], label = "CT"+str(telId))
            if telId == 4:
                #plt.xlim(100,200)
                plt.xlabel("Time from \"GetTime()\" [s]")
                plt.ylabel("NSB [MHz]")
                plt.legend()
                plt.show()

                fig = plt.figure(figsize=(12,6), )

        except:
            print("No CT"+str(telId)+" Data")
    plt.legend()
    plt.xlabel("Time from \"GetTime()\" [s]")
    #plt.xlim(117, 121)
    #plt.ylim(450, 470)
    plt.ylabel("NSB [MHz]")
    plt.show()
print(possible_peak_times)

In [ ]:
%%time
telId = 5
for nrun in nruns:
    plot_scatter(high_pixel[nrun]["x-pos"+str(telId)], 
                 high_pixel[nrun]["y-pos"+str(telId)], 
                 high_pixel[nrun]["tmp"+str(telId)]-high_pixel[nrun]["tmp"+str(telId)][0] )
    plt.show()
    print("test")

    #first cleaning:
    x_cleaned_1,y_cleaned_1,z_cleaned_1 = cleaning_cut(high_pixel[nrun]["x-pos"+str(telId)], 
                                                       high_pixel[nrun]["y-pos"+str(telId)], 
                                                       high_pixel[nrun]["tmp"+str(telId)]-high_pixel[nrun]["tmp"+str(telId)][0], 3, 3)

    #second cleaning
    result = cleaning_cut(x_cleaned_1, y_cleaned_1, z_cleaned_1, 4, 2)


    tracks = sort_into_tracks(result[0],result[1], result[2], 6)
    for N in range(1,6):
        if len(tracks["x-"+str(N)]) !=0:
            number_of_tracks +=1
            print("Track", N, end =": ")
            popt, pcov = curve_fit(lin_fit, tracks["x-"+str(N)], tracks["y-"+str(N)])
            print("linear fit : y =",round(popt[0],4),"* x +",round(popt[1],4)) 
            dur = tracks["z-"+str(N)][-1]- tracks["z-"+str(N)][0]
            print("Duration: ", int(dur), "s" )
            x_y_len = np.sqrt((tracks["x-"+str(N)][-1]-tracks["x-"+str(N)][0])**2+
                              (lin_fit(np.array(tracks["x-"+str(N)]), *popt)[-1] - 
                               lin_fit(np.array(tracks["x-"+str(N)]), *popt)[0])**2 
                             )
            print("Length:", round(x_y_len, 4), "m(?)")
            print("Speed on Cam: ", round(x_y_len/dur,4), "m/s")
            x_y = []
            for i in range(len(tracks["x-"+str(N)])):
                x_y.append((round(tracks["x-"+str(N)][i],4), 
                            round(tracks["y-"+str(N)][i],4)))
            print("Unique pixels:",len(list(set(x_y))))
            print("Pixels per unit of length:", round(len(list(set(x_y)))/x_y_len, 4) )
            plt.plot(np.array(tracks["x-"+str(N)]), lin_fit(np.array(tracks["x-"+str(N)]), *popt))
            plt.plot()
            plot_scatter(tracks["x-"+str(N)],tracks["y-"+str(N)], tracks["z-"+str(N)])
            plt.show()
    print("number of tracks: ", number_of_tracks)

In [ ]:
%%time
telId = 5
for nrun in nruns:
    
    brightness_hist0, brightness_counts = np.unique(np.around(high_pixel[nrun][telId],-1), return_counts=True)    
    plt.plot(brightness_hist0, brightness_counts)
    plt.title(nrun)
    plt.show()

In [ ]:
%%time
nrun = 158214
telId = 5
# for i in range(0,13):
#     mask = (high_pixel[nrun]["tmp"+str(telId)]-high_pixel[nrun]["tmp"+str(telId)][0])>i*30
#     mask = np.logical_and(mask,(high_pixel[nrun]["tmp"+str(telId)]-high_pixel[nrun]["tmp"+str(telId)][0])<(i+1)*30)
#     plt.scatter(high_pixel[nrun]["x-pos"+str(telId)][mask], high_pixel[nrun]["y-pos"+str(telId)][mask], 
#                 c = high_pixel[nrun]["tmp"+str(telId)][mask]-high_pixel[nrun]["tmp"+str(telId)][0], cmap = "jet")
#     plt.colorbar()
#     plt.show()
print(nruns)
avg_from_high_pixel = []
nruns_avg_from_high_pixel = []
for nrun in nruns:
    print(np.average(high_pixel[nrun][telId]))
    avg_from_high_pixel.append(np.average(high_pixel[nrun][telId]))
    nruns_avg_from_high_pixel.append(nrun)
plt.plot(nruns_avg_from_high_pixel, avg_from_high_pixel )
plt.show()

In [ ]:
%%time 
telId = 5
nrun = 158358
mask = np.logical_and(high_pixel[nrun][telId]<10000,high_pixel[nrun][telId]>2000 )
values, counts = np.unique(high_pixel[nrun]["pix"+str(telId)][mask], return_counts=True)
mask2 = counts<30
plt.scatter(values[mask2], counts[mask2])
plt.show()
for i in range(1,17):
    mask_time = np.logical_and((high_pixel[nrun]["tmp"+str(telId)][mask]-high_pixel[nrun]["tmp"+str(telId)][0])<100*(i+1),
                              (high_pixel[nrun]["tmp"+str(telId)][mask]-high_pixel[nrun]["tmp"+str(telId)][0])>100*i)
    
    plot_scatter(high_pixel[nrun]["x-pos"+str(telId)][mask][mask_time],
                 high_pixel[nrun]["y-pos"+str(telId)][mask][mask_time],
                 high_pixel[nrun]["tmp"+str(telId)][mask][mask_time]-high_pixel[nrun]["tmp"+str(telId)][0])

    plt.show()

In [ ]:
%%time
nrun = nruns[0]
telId = 5
time_in_run = high_pixel[nrun]["tmp"+str(telId)]-high_pixel[nrun]["tmp"+str(telId)][0]
test_x = []
test_y = []
test_z = []
test_z_1 = []
test_z_2 = []
values, counts = np.unique(high_pixel[nrun]["pix"+str(telId)], return_counts=True)
val_num = []
for i in range(len(values)):
    if counts[i]<30:
        #print(values[i], counts[i])
        val_num.append(values[i])
print(len(val_num))
for i in range(len(high_pixel[nrun]["x-pos"+str(telId)])):
    if high_pixel[nrun][telId][i]<3000:
        test_x.append(high_pixel[nrun]["x-pos"+str(telId)][i])
        test_y.append(high_pixel[nrun]["y-pos"+str(telId)][i])
        test_z_1.append(high_pixel[nrun][telId][i])
        test_z_2.append(time_in_run[i])
mask = np.array(test_z_2) < 345.
mask = np.logical_and(mask, np.array(test_z_2)> 320.)
print("Unique times:", np.unique(np.array(test_z_2)[mask]))
print("minimum",np.min(high_pixel[nrun][telId]))
print(len(high_pixel[nrun]["x-pos"+str(telId)]))
print(len(test_z_1))
plt.scatter(np.array(test_x)[mask],np.array(test_y)[mask],c=np.array(test_z_1)[mask])
plt.colorbar()
plt.show()
plot_scatter(np.array(test_x)[mask],np.array(test_y)[mask],np.array(test_z_2)[mask])
plt.show()
brightness_hist0, brightness_counts = np.unique(np.around(high_pixel[nrun][telId],0), return_counts=True)
plt.plot(brightness_hist0, brightness_counts)
plt.xlim(800,5000)
plt.show()
# x_cleaned_1,y_cleaned_1,z_cleaned_1 = cleaning_cut(high_pixel[nrun]["x-pos"+str(telId)], 
#                                                        high_pixel[nrun]["y-pos"+str(telId)], 
#                                                        time_in_run, 3, 3)
# plot_scatter(x_cleaned_1, y_cleaned_1, z_cleaned_1)
# plt.show()

In [ ]:
%%time
#Sort by pixels
tmp_from_high = {}
for nrun in nruns:
    tmp_from_high[nrun] = {}
    for telId in range(1,6):
        tmp_from_high[nrun][telId] = {}
        tmp_from_high[nrun][telId]["all pixels"] = {}
        try:
            print("Run", nrun,"CT"+str(telId)+":", high_pixel[nrun]["pix"+str(telId)])
            counter = 0
            for pix in high_pixel[nrun]["pix"+str(telId)]:
                if "tmp"+str(pix) in tmp_from_high[nrun][telId]:
                    pass
                else:
                    tmp_from_high[nrun][telId]["tmp"+str(pix)] = []
                    tmp_from_high[nrun][telId]["nsb"+str(pix)] = []
                    tmp_from_high[nrun][telId]["x-pos"+str(pix)] = []
                    tmp_from_high[nrun][telId]["y-pos"+str(pix)] = []
                    tmp_from_high[nrun][telId]["all pixels"][pix] = pix 
                tmp_from_high[nrun][telId]["tmp"+str(pix)].append(high_pixel[nrun]["tmp"+str(telId)][counter])
                tmp_from_high[nrun][telId]["nsb"+str(pix)].append(high_pixel[nrun][telId][counter])
                tmp_from_high[nrun][telId]["x-pos"+str(pix)].append(high_pixel[nrun]["x-pos"+str(telId)][counter])
                tmp_from_high[nrun][telId]["y-pos"+str(pix)].append(high_pixel[nrun]["y-pos"+str(telId)][counter])
                #print(pix,tmp_from_roll[nrun][telId]["tmp"+str(pix)], tmp_from_roll[nrun][telId]["nsb"+str(pix)])
                
                counter = counter+1
        except:
            print("Run", nrun,",CT", telId,"does not exist")



for nrun in nruns:
    print("")
    print("Run number", nrun)
    for telId in range(1,6):
        print("")
        print("Run", nrun,", CT", telId, end =": ")
        counter = 0
        for pix in sorted(tmp_from_high[nrun][telId]["all pixels"]):
            print(int(tmp_from_high[nrun][telId]["all pixels"][pix]), end = ", ")
            x = tmp_from_high[nrun][telId]["tmp"+str(pix)]
            y = tmp_from_high[nrun][telId]["nsb"+str(pix)]
            plt.plot(x,y, label = int(pix), lw = 0.5)
            counter+=1
            if counter >8:                
                plt.legend()
                plt.show()
                counter = 0
                print("Run", nrun,", CT", telId, end=": ")
        if counter >0:
            plt.legend()
            plt.show()
            


In [ ]:
%%time
nrun = 158212
telId = 4
new_x = []
new_y = []
new_time = []
time = high_pixel[nrun]["tmp"+str(telId)]-high_pixel[nrun]["tmp"+str(telId)][0]
lowlim = 1430
uplim = 1445
for i in range(len(high_pixel[nrun]["y-pos"+str(telId)])):
    if lowlim<time[i] <uplim:
        new_x.append(high_pixel[nrun]["x-pos"+str(telId)][i])
        new_y.append(high_pixel[nrun]["y-pos"+str(telId)][i])
        new_time.append(time[i])
plt.scatter(new_x, new_y, c = new_time, cmap = "jet")
plt.colorbar(label = "time in s")
plt.xlabel("x-coordinate")
plt.ylabel("y-coordinate")
plt.savefig(str(nrun)+" CT 5.pdf")
plt.show()
pix_for_now = []
for pix in sorted(tmp_from_high[nrun][telId]["all pixels"]):
    #print(int(tmp_from_high[nrun][telId]["all pixels"][pix]), end = ", ")
    tmp = tmp_from_high[nrun][telId]["tmp"+str(pix)]-high_pixel[nrun]["tmp"+str(telId)][0]
    nsb = tmp_from_high[nrun][telId]["nsb"+str(pix)]
    print("pixel", pix, end=": ")
    lower = np.where(lowlim<=tmp)
    upper = np.where(uplim>=tmp)
    try:
        print(list(tmp[lower[0]])[0], end ="") 
    except:
        pass
    try:
        print(list(tmp[upper[-1]])[-1])
    except:
        print("")
    if np.std(nsb)>500:
        print(pix, ", ",np.std(nsb))
        plt.plot(tmp,nsb, label = int(pix), lw = 0.5)
        pix_for_now.append(pix)
plt.xlim(lowlim,uplim)
plt.ylim(0,7000)
plt.show()
#print(pix_for_now)
newnew_x = []
newnew_y = []
print(len(pix_for_now))
for i in range(0,1):
    print(i)
#     pixel = pix_for_now[i]
#     newnew_x.append(geom_hess5_xc_from_root[pixel])
#     newnew_y.append(geom_hess5_yc_from_root[pixel])
#     print(pixel)
plt.plot(newnew_x, newnew_y)
plt.show()

In [ ]:


for nrun in nruns:
    print("")
    print("Run number", nrun)
    for telId in range(1,6):
        print("")
        print("CT", telId, end =": ")
        counter = 0
        for pix in sorted(tmp_from_roll[nrun][telId]["all pixels"]):
            print(int(tmp_from_roll[nrun][telId]["all pixels"][pix]), end = ", ")
            x = tmp_from_roll[nrun][telId]["tmp"+str(pix)]
            y = tmp_from_roll[nrun][telId]["nsb"+str(pix)]
            plt.scatter(x,y, label = int(pix))
            counter+=1
            if counter >8:                
                plt.legend()
                plt.show()
                counter = 0
                print("CT", telId, end=": ")
        if counter >0:
            plt.legend()
            plt.show()

In [ ]:
%%time
telId = 5
nrun = nruns[1]
print(len(high_pixel[nrun]["x-pos"+str(telId)]))
plot_scatter(high_pixel[nrun]["x-pos"+str(telId)],
            high_pixel[nrun]["y-pos"+str(telId)],
            high_pixel[nrun]["tmp"+str(telId)])
draw_box()
plt.show()

In [ ]:
%%time
# Remove all pixels that have more than max_counts occurences, since those are most of the time no satellite trails.
# losses along a satellite trail have to be accepted. Max_counts is deliberately chosen generously
#################Old version#################
#################Do Not Use this one#########
trails_high_avg = {}
telId = 5
max_counts = 50

for nrun in nruns:
    trails_high_avg[nrun] = {}
    for telId in range(5,6):
        trails_high_avg[nrun][telId] = {}
        try:
            values, counts = np.unique(high_pixel[nrun]["pix"+str(telId)], return_counts=True)
            print("Length of values", len(values))
            trails_high_avg[nrun][telId]["unique pix ID"] = []
            trails_high_avg[nrun][telId]["pix ID counts"] = []
            new_val = []
            new_counts = []
            for i in range(len(values)):
                if counts[i]<max_counts:#Number needs to be calculated for slowest satellite
                    new_val.append(values[i])
                    new_counts.append(counts[i])

                    trails_high_avg[nrun][telId]["unique pix ID"].append(values[i])
                    trails_high_avg[nrun][telId]["pix ID counts"].append(counts[i])

            trails_high_avg[nrun][telId]["pix ID"] = []
            trails_high_avg[nrun][telId]["x"] = []
            trails_high_avg[nrun][telId]["y"] = []
            trails_high_avg[nrun][telId]["z"] = []
            trails_high_avg[nrun][telId]["brightness"] = []

            print("Run", nrun, ", CT", telId,":")
            for i in range(len(high_pixel[nrun]["pix"+str(telId)])):
                if high_pixel[nrun]["pix"+str(telId)][i] in trails_high_avg[nrun][telId]["unique pix ID"]:

                    if high_pixel[nrun]["tmp"+str(telId)][i]-high_pixel[nrun]["tmp"+str(telId)][0] <5:
                        pass

                    else:
                        trails_high_avg[nrun][telId]["pix ID"].append(int(high_pixel[nrun]["pix"+str(telId)][i]))
                        trails_high_avg[nrun][telId]["x"].append(high_pixel[nrun]["x-pos"+str(telId)][i])
                        trails_high_avg[nrun][telId]["y"].append(high_pixel[nrun]["y-pos"+str(telId)][i])
                        trails_high_avg[nrun][telId]["z"].append(high_pixel[nrun]["tmp"+str(telId)][i]-high_pixel[nrun]["tmp"+str(telId)][0])
                        trails_high_avg[nrun][telId]["brightness"].append(high_pixel[nrun][telId][i])

            plt.scatter(trails_high_avg[nrun][telId]["x"],
                        trails_high_avg[nrun][telId]["y"],
                        c=trails_high_avg[nrun][telId]["z"], 
                        cmap = "jet")
            plt.colorbar()
            plt.xlim(-1.2,1.2)
            plt.ylim(-1.2,1.2)
            plt.show()
        except:
            pass

Plan: GetNeighboursList.C schreiben, um PixId von neighbours and nextneigbours zu erhalten. 
=> Beschleunigung von cleaning

width of trail (done),

angular distance, duration (done to second timescale), speed (m/s, pixels/s(done) )

direction of run (zenith) (still need code snippet to insert in C++ )

CT1-4 cuts, finding trails

In [ ]:
%%time
#Version for only CT5
#Remove all pixels that have too few neighbouring pixels by cleaning twice
telId = 5
x_values = np.arange(-1.2,1.2, 0.01)
for nrun in nruns:
    try:
        print("CT", telId, ", Run", nrun, ":")
        plot_scatter(trails_high_avg[nrun][telId]["x"], trails_high_avg[nrun][telId]["y"], 
                    trails_high_avg[nrun][telId]["z"])    
        plt.show()
        #first  cleaning:
        x_cleaned_1,y_cleaned_1,z_cleaned_1, brightness_cleaned_1 = cleaning_cut(trails_high_avg[nrun][telId]["x"],
                                                                                 trails_high_avg[nrun][telId]["y"],
                                                                                 trails_high_avg[nrun][telId]["z"],
                                                                                 trails_high_avg[nrun][telId]["brightness"],4, 3)

        #second cleaning
        result = cleaning_cut(x_cleaned_1, y_cleaned_1, z_cleaned_1, brightness_cleaned_1, 4, 3)
        trails_high_avg[nrun][telId]["x_cleaned"] = result[0]
        trails_high_avg[nrun][telId]["y_cleaned"] = result[1]
        trails_high_avg[nrun][telId]["z_cleaned"] = result[2]
        trails_high_avg[nrun][telId]["brightness_cleaned"] = result[3]

        tracks, meterorites = sort_into_tracks(trails_high_avg[nrun][telId]["x_cleaned"],
                                               trails_high_avg[nrun][telId]["y_cleaned"],
                                               trails_high_avg[nrun][telId]["z_cleaned"], 
                                               trails_high_avg[nrun][telId]["brightness_cleaned"], 6)
        trails_high_avg[nrun][telId]["tracks"] = tracks
        print("Trying to separate:")
        number_of_tracks = 0
        for N in range(1,6):
            if len(tracks["x-"+str(N)]) !=0:
                number_of_tracks +=1
                trails_high_avg[nrun][telId]["tracks"]["x-"+str(number_of_tracks)] = trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)]
                trails_high_avg[nrun][telId]["tracks"]["y-"+str(number_of_tracks)] = trails_high_avg[nrun][telId]["tracks"]["y-"+str(N)]
                trails_high_avg[nrun][telId]["tracks"]["z-"+str(number_of_tracks)] = trails_high_avg[nrun][telId]["tracks"]["z-"+str(N)]
                trails_high_avg[nrun][telId]["tracks"]["brightness-"+str(number_of_tracks)] = np.around(trails_high_avg[nrun][telId]["tracks"]["brightness-"+str(N)],1)
                if N != number_of_tracks:
                    trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)] = []
                    trails_high_avg[nrun][telId]["tracks"]["y-"+str(N)] = []
                    trails_high_avg[nrun][telId]["tracks"]["z-"+str(N)] = []
                    trails_high_avg[nrun][telId]["tracks"]["brightness-"+str(N)] = []
                print("Track", number_of_tracks, end =": ")
                popt, pcov = curve_fit(lin_fit, tracks["x-"+str(number_of_tracks)], tracks["y-"+str(number_of_tracks)])
                print("linear fit : y =",round(popt[0],4),"* x +",round(popt[1],4)) 
                dur = tracks["z-"+str(number_of_tracks)][-1]- tracks["z-"+str(number_of_tracks)][0]
                print("Duration: ", int(dur), "s" )
                x_y_len = np.sqrt((tracks["x-"+str(number_of_tracks)][-1]-tracks["x-"+str(number_of_tracks)][0])**2+
                                  (lin_fit(np.array(tracks["x-"+str(number_of_tracks)]), *popt)[-1] - 
                                   lin_fit(np.array(tracks["x-"+str(number_of_tracks)]), *popt)[0])**2 
                                 )
                print("Length:", round(x_y_len, 4), "m(?)")
                print("Average brightness:", round(np.average(tracks["brightness-"+str(number_of_tracks)]),1) , "+-", 
                      round(np.std(tracks["brightness-"+str(number_of_tracks)]), 1))
                print("Speed on Cam: ", round(x_y_len/dur,4), "m/s")
                x_y = []
                for i in range(len(tracks["x-"+str(number_of_tracks)])):
                    x_y.append((round(tracks["x-"+str(number_of_tracks)][i],4), 
                                round(tracks["y-"+str(number_of_tracks)][i],4)))
                print("Unique pixels:",len(list(set(x_y))))
                print("Pixels per unit of length:", round(len(list(set(x_y)))/x_y_len, 4) )             
                get_track_width(trails_high_avg[nrun][telId]["tracks"]["x-"+str(number_of_tracks)],
                               trails_high_avg[nrun][telId]["tracks"]["y-"+str(number_of_tracks)])
                #intersection = draw_box(_new(popt[0], popt[1]))
                trails_high_avg[nrun][telId]["tracks"]["IS track "+str(number_of_tracks)] = []
                trails_high_avg[nrun][telId]["tracks"]["IS track "+str(number_of_tracks)].append(intersection)
                plt.plot(np.array(tracks["x-"+str(number_of_tracks)]), lin_fit(np.array(tracks["x-"+str(number_of_tracks)]), *popt))
                plot_scatter(tracks["x-"+str(number_of_tracks)],tracks["y-"+str(number_of_tracks)], tracks["z-"+str(number_of_tracks)])
    #           '  
    #             print_important_params(tracks["x-"+str(N)],
    #                                    tracks["y-"+str(N)],
    #                                    tracks["z-"+str(N)])'

                plt.savefig("run_"+str(nrun)+"_CT_"+str(telId)+"_track_"+str(number_of_tracks)+".pdf")
                plt.show()
                brightness_hist0, brightness_counts = np.unique(np.around(tracks["brightness-"+str(number_of_tracks)],-3), return_counts=True)
                plt.plot(brightness_hist0, brightness_counts)
                plt.show()


        print("Number of satellite tracks found in run", nrun, "CT", telId, ": ", number_of_tracks)
        print("Number of tracks identified as metiorites:", len(meterorites.keys()))
    except:
        print("No CT 5 data available")

In [ ]:
%%time
#version without cleaning first
telId = 5 
x_values = np.arange(-1.2,1.2, 0.01)
for nrun in nruns:
    try:
        print("CT", telId, ", Run", nrun, ":")
        plot_scatter(trails_high_avg[nrun][telId]["x"], trails_high_avg[nrun][telId]["y"], 
                    trails_high_avg[nrun][telId]["z"])    
        plt.show()
        tracks, meterorites = sort_into_tracks(trails_high_avg[nrun][telId]["x"],
                                               trails_high_avg[nrun][telId]["y"],
                                               trails_high_avg[nrun][telId]["z"], 
                                               trails_high_avg[nrun][telId]["brightness"], 6)
        trails_high_avg[nrun][telId]["tracks"] = tracks
        print("Trying to separate:")
        number_of_tracks = 0
        for N in range(1,9):
            if len(tracks["x-"+str(N)]) !=0:
                number_of_tracks +=1
                trails_high_avg[nrun][telId]["tracks"]["x-"+str(number_of_tracks)] = trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)]
                trails_high_avg[nrun][telId]["tracks"]["y-"+str(number_of_tracks)] = trails_high_avg[nrun][telId]["tracks"]["y-"+str(N)]
                trails_high_avg[nrun][telId]["tracks"]["z-"+str(number_of_tracks)] = trails_high_avg[nrun][telId]["tracks"]["z-"+str(N)]
                trails_high_avg[nrun][telId]["tracks"]["brightness-"+str(number_of_tracks)] = np.around(trails_high_avg[nrun][telId]["tracks"]["brightness-"+str(N)],1)
                if N != number_of_tracks:
                    trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)] = []
                    trails_high_avg[nrun][telId]["tracks"]["y-"+str(N)] = []
                    trails_high_avg[nrun][telId]["tracks"]["z-"+str(N)] = []
                    trails_high_avg[nrun][telId]["tracks"]["brightness-"+str(N)] = []
                print("Track", number_of_tracks, end =": ")
                popt, pcov = curve_fit(lin_fit, tracks["x-"+str(number_of_tracks)], tracks["y-"+str(number_of_tracks)])
                print("linear fit : y =",round(popt[0],4),"* x +",round(popt[1],4)) 
                dur = tracks["z-"+str(number_of_tracks)][-1]- tracks["z-"+str(number_of_tracks)][0]
                print("Duration: ", int(dur), "s" )
                x_y_len = np.sqrt((tracks["x-"+str(number_of_tracks)][-1]-tracks["x-"+str(number_of_tracks)][0])**2+
                                  (lin_fit(np.array(tracks["x-"+str(number_of_tracks)]), *popt)[-1] - 
                                   lin_fit(np.array(tracks["x-"+str(number_of_tracks)]), *popt)[0])**2 
                                 )
                print("Length:", round(x_y_len, 4), "m(?)")
                print("Average brightness:", round(np.average(tracks["brightness-"+str(number_of_tracks)]),1) , "+-", 
                      round(np.std(tracks["brightness-"+str(number_of_tracks)]), 1))
                print("Speed on Cam: ", round(x_y_len/dur,4), "m/s")
                x_y = []
                for i in range(len(tracks["x-"+str(number_of_tracks)])):
                    x_y.append((round(tracks["x-"+str(number_of_tracks)][i],4), 
                                round(tracks["y-"+str(number_of_tracks)][i],4)))
                print("Unique pixels:",len(list(set(x_y))))
                print("Pixels per unit of length:", round(len(list(set(x_y)))/x_y_len, 4) )             
                get_track_width(trails_high_avg[nrun][telId]["tracks"]["x-"+str(number_of_tracks)],
                               trails_high_avg[nrun][telId]["tracks"]["y-"+str(number_of_tracks)])
                intersection = draw_box_new(popt[0], popt[1])
                trails_high_avg[nrun][telId]["tracks"]["IS track "+str(number_of_tracks)] = []
                trails_high_avg[nrun][telId]["tracks"]["IS track "+str(number_of_tracks)].append(intersection)
                plt.plot(np.array(tracks["x-"+str(number_of_tracks)]), lin_fit(np.array(tracks["x-"+str(number_of_tracks)]), *popt))
                plot_scatter(tracks["x-"+str(number_of_tracks)],tracks["y-"+str(number_of_tracks)], tracks["z-"+str(number_of_tracks)])
                plt.show()
        print("Number of satellite tracks found in run", nrun, "CT", telId, ": ", number_of_tracks)
        print("Number of tracks identified as metiorites:", len(meterorites.keys()))
    except:
        print("No CT 5 data available")

In [ ]:
%%time
#Version for all CTs
#Remove all pixels that have too few neighbouring pixels by cleaning twice
telId = 5
x_values = np.arange(-1.2,1.2, 0.01)
for nrun in nruns:
    for telId in range(5,6):
        try:
            print("CT", telId, ", Run", nrun, ":")
            plot_scatter(trails_high_avg[nrun][telId]["x"], trails_high_avg[nrun][telId]["y"], 
                        trails_high_avg[nrun][telId]["z"])    
            plt.show()
            #first  cleaning:
            x_cleaned_1,y_cleaned_1,z_cleaned_1, brightness_cleaned_1 = cleaning_cut(trails_high_avg[nrun][telId]["x"],
                                                                                     trails_high_avg[nrun][telId]["y"],
                                                                                     trails_high_avg[nrun][telId]["z"],
                                                                                     trails_high_avg[nrun][telId]["brightness"],3, 3)

            #second cleaning
            result = cleaning_cut(x_cleaned_1, y_cleaned_1, z_cleaned_1, brightness_cleaned_1, 4, 2)
            trails_high_avg[nrun][telId]["x_cleaned"] = result[0]
            trails_high_avg[nrun][telId]["y_cleaned"] = result[1]
            trails_high_avg[nrun][telId]["z_cleaned"] = result[2]
            trails_high_avg[nrun][telId]["brightness_cleaned"] = result[3]

            tracks, meterorites = sort_into_tracks(trails_high_avg[nrun][telId]["x_cleaned"],
                                                   trails_high_avg[nrun][telId]["y_cleaned"],
                                                   trails_high_avg[nrun][telId]["z_cleaned"], 
                                                   trails_high_avg[nrun][telId]["brightness_cleaned"], 6)
            trails_high_avg[nrun][telId]["tracks"] = tracks
            print("Trying to separate:")
            number_of_tracks = 0
            for N in range(1,6):
                if len(tracks["x-"+str(N)]) !=0:
                    number_of_tracks +=1
                    print("Track", N, end =": ")
                    popt, pcov = curve_fit(lin_fit, tracks["x-"+str(N)], tracks["y-"+str(N)])
                    print("linear fit : y =",round(popt[0],4),"* x +",round(popt[1],4)) 
                    dur = tracks["z-"+str(N)][-1]- tracks["z-"+str(N)][0]
                    print("Duration: ", int(dur), "s" )
                    x_y_len = np.sqrt((tracks["x-"+str(N)][-1]-tracks["x-"+str(N)][0])**2+
                                      (lin_fit(np.array(tracks["x-"+str(N)]), *popt)[-1] - 
                                       lin_fit(np.array(tracks["x-"+str(N)]), *popt)[0])**2 
                                     )
                    print("Length:", round(x_y_len, 4), "m(?)")
                    print("Average brightness:", round(np.average(tracks["brightness-"+str(N)]),1) , "+-", 
                          round(np.std(tracks["brightness-"+str(N)]), 1))
                    print("Speed on Cam: ", round(x_y_len/dur,4), "m/s")
                    x_y = []
                    for i in range(len(tracks["x-"+str(N)])):
                        x_y.append((round(tracks["x-"+str(N)][i],4), 
                                    round(tracks["y-"+str(N)][i],4)))
                    print("Unique pixels:",len(list(set(x_y))))
                    print("Pixels per unit of length:", round(len(list(set(x_y)))/x_y_len, 4) )             
                    get_track_width(trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)],
                                   trails_high_avg[nrun][telId]["tracks"]["y-"+str(N)])
                    intersection = draw_box_new(popt[0], popt[1])
                    trails_high_avg[nrun][telId]["tracks"]["IS track "+str(number_of_tracks)] = []
                    trails_high_avg[nrun][telId]["tracks"]["IS track "+str(number_of_tracks)].append(intersection)
                    plt.plot(np.array(tracks["x-"+str(N)]), lin_fit(np.array(tracks["x-"+str(N)]), *popt))
                    plot_scatter(tracks["x-"+str(N)],tracks["y-"+str(N)], tracks["z-"+str(N)])
        #           '  
        #             print_important_params(tracks["x-"+str(N)],
        #                                    tracks["y-"+str(N)],
        #                                    tracks["z-"+str(N)])'

                    plt.savefig("run_"+str(nrun)+"_CT_"+str(telId)+"_track_"+str(number_of_tracks)+".pdf")
                    plt.show()

            print("Number of satellite tracks found in run", nrun, "CT", telId, ": ", number_of_tracks)
            print("Number of tracks identified as metiorites:", len(meterorites.keys()))
        except:
            pass

In [ ]:
nrun = 158385
telId = 5
print(high_pixel.keys())
mask = np.array(trails_high_avg[nrun][telId]["z"]) < 800.
mask = np.logical_and(mask, np.array(trails_high_avg[nrun][telId]["z"])>790.)
print(mask)
plot_scatter(np.array(trails_high_avg[nrun][telId]["x"])[mask],
             np.array(trails_high_avg[nrun][telId]["y"])[mask], 
             np.array(trails_high_avg[nrun][telId]["z"])[mask])
print(np.array(trails_high_avg[nrun][telId]["z"])[mask])
plt.show()

In [ ]:
%%time
nrun = 158229
telId = 5
#print(high_pixel[nrun].keys())
print(len(high_pixel[nrun][telId]))
mask = high_pixel[nrun][telId] >200. 
print(len(np.where(mask==True)[0]))
mask = np.logical_and(mask, high_pixel[nrun][telId] <5000)
print(len(np.where(mask==True)[0]))
#mask = np.logical_and(mask, high_pixel[nrun]["tmp"+str(telId)]-high_pixel[nrun]["tmp"+str(telId)][0]>350)
print(len(np.where(mask==True)[0]))
plot_scatter(high_pixel[nrun]["x-pos"+str(telId)][mask], high_pixel[nrun]["y-pos"+str(telId)][mask],
             high_pixel[nrun]["tmp"+str(telId)][mask]-high_pixel[nrun]["tmp"+str(telId)][0])
if telId != 5:
    plt.xlim(-0.75,0.75)
    plt.ylim(-0.75,0.75)
plt.show()

get charateristic parameters for each run which are of interest:
nrun, telid, direction (zenith, azimuth or sky coordinates RA, DEC), trail direction (maybe in appropriate coordinate system, 
duration, start-/end-time in UTC, mean brightness of trail, 

applying mask of CT5 on CT1-4 

mon 11:00

In [ ]:
nrun = 158229
telId = 5
for N in range(1,6):
    if len(trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)])!=0:
        popt, pcov = curve_fit(lin_fit, trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)], 
                              trails_high_avg[nrun][telId]["tracks"]["y-"+str(N)])
        print("linear fit : y =",round(popt[0],4),"* x +",round(popt[1],4)) 
        plot_scatter(trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)],
                     trails_high_avg[nrun][telId]["tracks"]["y-"+str(N)],
                     trails_high_avg[nrun][telId]["tracks"]["z-"+str(N)])
        
        plt.plot(np.array(trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)]), lin_fit(np.array(trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)]), *popt))
        
        d_from_lin_fit = []
        for i in range(len(trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)])):
            x_p = trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)][i]
            y_p = trails_high_avg[nrun][telId]["tracks"]["y-"+str(N)][i]
            t_2 = x_p/popt[0]+y_p
            x_s = popt[0]/(popt[0]**2+1)*(t_2 -popt[1])
            y_s = popt[0]*x_s+popt[1]
            d_from_lin_fit.append(np.sqrt(((x_p-x_s)/min_x_dif)**2+((y_p-y_s)/min_y_dif)**2))
        print(np.average(d_from_lin_fit))
        get_track_width(trails_high_avg[nrun][telId]["tracks"]["x-"+str(N)], 
                       trails_high_avg[nrun][telId]["tracks"]["y-"+str(N)])
        draw_box(popt[0], popt[1])
        plt.show()

In [ ]:
#linear fits, direction of travel and speed
#popt, pcov = curve_fit(lin_fit, track_x_1, track_y_1)
#vector:
# v =[track_x_1[-1]-track_x_1[0],track_y_1[-1]-track_y_1[-0]]

In [ ]:
%%time 
nrun = 158229
print("CT", telId, ", Run", nrun, ":")
plt.scatter(trails_high_avg[nrun][telId]["x_cleaned"], trails_high_avg[nrun][telId]["y_cleaned"],
           c= trails_high_avg[nrun][telId]["z_cleaned"], cmap = "jet")
plt.xlim(-1.2,1.2)
plt.ylim(-1.2,1.2)
plt.colorbar()
plt.show()
track_x_1 = []
track_x_2 = []
track_y_1 = []
track_y_2 = []
track_z_1 = []
track_z_2 = []
for i in range(len(trails_high_avg[nrun][telId]["x_cleaned"])):      
    x = trails_high_avg[nrun][telId]["x_cleaned"][i]
    y = trails_high_avg[nrun][telId]["y_cleaned"][i]
    z = trails_high_avg[nrun][telId]["z_cleaned"][i]
    #neighbours and next neighbours
    nn_x_xcut = [] 
    nn_y_xcut = []
    nn_z_xcut = []
    nn_x = [] 
    nn_y = []
    nn_z = []
    for j in range(len(trails_high_avg[nrun][telId]["x_cleaned"])):
        if (x-2*min_x_dif<
            trails_high_avg[nrun][telId]["x_cleaned"][j] < 
            x + 2*min_x_dif):
            nn_x_xcut.append(trails_high_avg[nrun][telId]["x_cleaned"][j])
            nn_y_xcut.append(trails_high_avg[nrun][telId]["y_cleaned"][j])
            nn_z_xcut.append(trails_high_avg[nrun][telId]["z_cleaned"][j])
    for j in range(len(nn_x_xcut)):
        if (y-2*min_y_dif<
            nn_y_xcut[j] < 
            y + 2*min_y_dif):
            nn_x.append(nn_x_xcut[j])
            nn_y.append(nn_y_xcut[j])
            nn_z.append(nn_z_xcut[j])
    
    if np.max(nn_x)-np.min(nn_x)> 4*min_x_dif:
        print(np.max(nn_x)-np.min(nn_x), "max allowed: ", 4*min_x_dif)
    if np.max(nn_y)-np.min(nn_y)> 4*min_y_dif:
        print(np.max(nn_y)-np.min(nn_y), "max allowed: ", 4*min_y_dif)
#     if i == 563:
#         plt.scatter(nn_x,nn_y, c = nn_z)
#         plt.xlim(-1.2,1.2)
#         plt.ylim(-1.2,1.2)
#         plt.colorbar()
#         plt.show()
    # Insert into tracks
    if i == 0:
        track_x_1.append(x)
        track_y_1.append(y)
        track_z_1.append(z)
    else:
        if track_x_1[-1] in nn_x and track_y_1[-1] in nn_y:
            track_x_1.append(x)
            track_y_1.append(y)
            track_z_1.append(z)
        elif len(track_x_2) == 0:
            track_x_2.append(x)
            track_y_2.append(y)
            track_z_2.append(z)
        elif track_x_2[-1] in nn_x and track_y_2[-1] in nn_y:
            track_x_2.append(x)
            track_y_2.append(y)
            track_z_2.append(z)
            
            
            
        
        
if len(track_x_1)!=0:      
    print("Trying to separate:")        
    plt.scatter(track_x_1, track_y_1, c = track_z_1, cmap = "jet")
    plt.xlim(-1.2,1.2)
    plt.ylim(-1.2,1.2)
    plt.colorbar()
    plt.arrow(track_x_1[0],track_y_1[0], track_x_1[-1]-track_x_1[0],track_y_1[-1]-track_y_1[0])
    plt.show()
    print("Time difference:", track_z_1[-1]-track_z_1[0], "s")
    print("Maximum distance on cam:", round(np.sqrt((track_x_1[-1]-track_x_1[0])**2 +(track_y_1[-1]-track_y_1[0])**2 ),3))
    if len(np.unique(track_z_1)) <3:
        print("Probably metiorite")
        print("")
print(len(track_x_1))
plt.plot(track_x_1[540:580], track_y_1[540:580])
plt.xlim(-1.2,1.2)
plt.ylim(-1.2,1.2)
plt.show()
print(len(track_x_2))

In [ ]:
a = [1,1,1,4,5]
b = [2,1,2,1,4]
a_b = []
for i in range(len(a)):
    a_b.append((a[i], b[i]))
print(a_b)
print(len(list(set(a_b))))

In [ ]:
x_fov = 5
y_fov = 5
x_fov_pp = x_fov/len(np.unique(geom_hess5_xc_from_root)) #per pixel
y_fov_pp = y_fov/len(np.unique(geom_hess5_yc_from_root))
x_fov_px = x_fov/(np.max(geom_hess5_xc_from_root)-np.min(geom_hess5_xc_from_root))
y_fov_py = y_fov/(np.max(geom_hess5_yc_from_root)-np.min(geom_hess5_yc_from_root))
print(x_fov_pp, y_fov_pp)
print(x_fov_px, y_fov_py)

In [ ]:
%%time
#reading zenith files:
os.chdir("D:\\Masterarbeit ECAP\\First plots")
az_zen_nrun = np.loadtxt("az_zen_file.txt", usecols=0, delimiter = ";")
az_zen_telId = np.loadtxt("az_zen_file.txt", usecols=1, delimiter = ";")
az_zen_az = np.loadtxt("az_zen_file.txt", usecols=2, delimiter = ";")
az_zen_zen = np.loadtxt("az_zen_file.txt", usecols=3, delimiter = ";")
az_zen_utc = np.genfromtxt("az_zen_file.txt", dtype=str, usecols=4, delimiter = ";")
os.chdir("D:\\Masterarbeit ECAP\\First plots\eval_data")

In [ ]:
%%time
#write to a file the interesting values such as 
#nrun, telId, az, zen, utc,
#1.track IS times and points and mean brigtness, 2. track IS times and points and mean brigtness, ... 
nrun = nruns[1]
print(nrun)
s = az_zen_utc[np.where(az_zen_nrun ==nrun)[0]][0]
print(s[6:])
header = ["nrun", "telId", "az", "zen", "utc first event in nrun",
          "1. track X1 ", "1. track Y1", "1. track X2", "1. track Y2", 
          "1. track time", "1. track duration", "1. track mean brightness",
          "2. track X1 ", "2. track Y1", "2. track X2", "2. track Y2", 
          "2. track time", "2. track duration", "2. track mean brightness",
          "3. track X1 ", "3. track Y1", "3. track X2", "3. track Y2", 
          "3. track time", "3. track duration", "3. track mean brightness"]
file_params = open("parameters.csv", "w")
params_writer = csv.writer(file_params)
params_writer.writerow(header) 
for i in range(len(az_zen_nrun)):
    row = [int(az_zen_nrun[i]), int(az_zen_telId[i]), int(az_zen_az[i]), int(az_zen_zen[i]), az_zen_utc[i]]
    for nrun in nruns:
        for telId in range(1,6):

            try:
                
                if (nrun == int(az_zen_nrun[i]) and int(az_zen_telId[i]) == telId):
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 1"][0][0][0])
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 1"][0][0][1])
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 1"][0][1][0])
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 1"][0][1][1])
                    print(trails_high_avg[nrun][telId]["tracks"]["IS track 1"][0][1][1])
                    row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-1"][0]))
                    row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-1"][-1] - 
                               trails_high_avg[nrun][telId]["tracks"]["z-1"][0]))
                    row.append(round(np.average(trails_high_avg[nrun][telId]["tracks"]["brightness-1"]),1))
                    try:
                        row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 2"][0][0][0])
                        row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 2"][0][0][1])
                        row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 2"][0][1][0])
                        row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 2"][0][1][1])
                        row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-2"][0]))
                        row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-2"][-1] - 
                                       trails_high_avg[nrun][telId]["tracks"]["z-2"][0]))
                        row.append(round(np.average(trails_high_avg[nrun][telId]["tracks"]["brightness-2"]),1))
                    except:
                        pass
                    try:
                        row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 3"][0][0][0])
                        row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 3"][0][0][1])
                        row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 3"][0][1][0])
                        row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 3"][0][1][1])
                        row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-3"][0]))
                        row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-3"][-1] - 
                                       trails_high_avg[nrun][telId]["tracks"]["z-3"][0]))
                        row.append(round(np.average(trails_high_avg[nrun][telId]["tracks"]["brightness-3"]),1))
                    except:
                        pass
            except:
                pass
            
    params_writer.writerow(row)
file_params.close()

In [ ]:
%%time
#write to a file the interesting values such as 
#nrun, telId, az, zen, utc,
#1.track IS times and points and mean brigtness, 2. track IS times and points and mean brigtness, ... 
nrun = nruns[1]
print(nrun)
s = az_zen_utc[np.where(az_zen_nrun ==nrun)[0]][0]
print(s[6:])
header_CT5 = ["nrun", "telId", "az", "zen", "utc first event in nrun",
          "1. track X1 ", "1. track Y1", "1. track X2", "1. track Y2", 
          "1. track time", "1. track duration", "1. track mean brightness",
          "2. track X1 ", "2. track Y1", "2. track X2", "2. track Y2", 
          "2. track time", "2. track duration", "2. track mean brightness",
          "3. track X1 ", "3. track Y1", "3. track X2", "3. track Y2", 
          "3. track time", "3. track duration", "3. track mean brightness"]
file_params_CT5 = open("parameters_CT5.csv", "w")
params_CT5_writer = csv.writer(file_params_CT5)
params_CT5_writer.writerow(header_CT5) 
row = []
for i in range(len(az_zen_nrun)):
    lastrow = row
    row = [int(az_zen_nrun[i]),int(az_zen_telId[i]), int(az_zen_az[i]), int(az_zen_zen[i]), az_zen_utc[i]]
    for nrun in nruns:
        try:

            if (nrun == int(az_zen_nrun[i]) and int(az_zen_telId[i]) == 5):
                print(nrun , int(az_zen_nrun[i]) , int(az_zen_telId[i]) , 5)
                row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 1"][0][0][0])
                row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 1"][0][0][1])
                row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 1"][0][1][0])
                row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 1"][0][1][1])
                print(trails_high_avg[nrun][telId]["tracks"]["IS track 1"][0][1][1])
                row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-1"][0]))
                row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-1"][-1] - 
                           trails_high_avg[nrun][telId]["tracks"]["z-1"][0]))
                row.append(round(np.average(trails_high_avg[nrun][telId]["tracks"]["brightness-1"]),1))
                try:
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 2"][0][0][0])
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 2"][0][0][1])
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 2"][0][1][0])
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 2"][0][1][1])
                    row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-2"][0]))
                    row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-2"][-1] - 
                                   trails_high_avg[nrun][telId]["tracks"]["z-2"][0]))
                    row.append(round(np.average(trails_high_avg[nrun][telId]["tracks"]["brightness-2"]),1))
                except:
                    pass
                try:
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 3"][0][0][0])
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 3"][0][0][1])
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 3"][0][1][0])
                    row.append(trails_high_avg[nrun][telId]["tracks"]["IS track 3"][0][1][1])
                    row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-3"][0]))
                    row.append(int(trails_high_avg[nrun][telId]["tracks"]["z-3"][-1] - 
                                   trails_high_avg[nrun][telId]["tracks"]["z-3"][0]))
                    row.append(round(np.average(trails_high_avg[nrun][telId]["tracks"]["brightness-3"]),1))
                except:
                    pass
                
        except:
            pass
    try:
        if row[1] ==5:
            print(row)
            params_CT5_writer.writerow(row)
    except:
        pass
file_params_CT5.close()

Old functions, that might become useful in some way later

In [ ]:
@jit
def trail_finder_no_dict(dict_pix, dict_tmp, dict_brightness):   
    if (len(dict_pix)==0):
        print("Too few values in run", nrun)
        return 
#     x = []
#     y = []
#     for i in range(len(pix)):
#         x.append(flash_geom_x[pix[i]])
#         y.append(flash_geom_y[pix[i]])##there must be a better way...
    pix = [dict_pix[0]]
    tmp = [dict_tmp[0]]
    brightness = [dict_brightness[0]]
    for i in range(1, len(dict_pix)):
        mask_time = np.logical_and( dict_tmp>dict_tmp[i]-5,dict_tmp<dict_tmp[i]+5)
        st = set(nn_pix[int(dict_pix[i])])
        mask_space = [i for i, e in enumerate(dict_pix[mask_time]) if e in st]
        
        if len(set(dict_pix[mask_time][mask_space]))>0:
            pix.append(dict_pix[i])
            tmp.append(dict_tmp[i])
            brightness.append(dict_brightness[i])
    tracks = [[[pix[0]], [tmp[0]], [brightness[0]]]]
#     tracks.append([])
#     tracks[0].append([pix[0]])
#     tracks[0].append([tmp[0]])
#     tracks[0].append([brightness[0]])
    N_tot = 1
    print(len(pix))
    for i in range(1, len(pix)):
        appended_to_track = False
        for N in range(len(tracks)):
            if len(set(tracks[N][0][-20:]) & set(nn_pix[pix[i]]))>0:
                if tmp[i]-5 < tracks[N][1][-1]:
                    tracks[N][0].append(pix[i])
                    tracks[N][1].append(tmp[i])
                    tracks[N][2].append(brightness[i])
                    appended_to_track  = True 
                    break
#             else:
#                 if len(tracks[N]["tmp"])<2:
#                     tracks.pop(N, None)
        if appended_to_track == False:
            tracks.append([[pix[i]], [tmp[i]], [brightness[i]]])
#             tracks.append([])
#             tracks[-1].append([pix[i]])
#             tracks[-1].append([tmp[i]])
#             tracks[-1].append([brightness[i]])
            continue
#     for N in range(len(tracks)):
#         if len(tracks[N][0]) < 3:
#             del(tracks[N])
    possible_meteorites = np.zeros(len(tracks))
    for N in range(len(tracks)):
        if len(set(tracks[N][0]))<3:
            print("Track", N, "from", set(tracks[N][1]),
                  ",too few individual pixels", len(set(tracks[N][0])))
            possible_meteorites[N] = [tracks[N][0], tracks[N][1], tracks[N][2]]
#             possible_meteorites.append([])
#             possible_meteorites[-1].append([tracks[N][0]])
#             possible_meteorites[-1].append([tracks[N][1]])
#             possible_meteorites[-1].append([tracks[N][2]])
        elif len(set(tracks[N][1]))<3:
            print("Track", N, "from", set(tracks[N][1]),
                  ",too few different times", set(tracks[N][1]))
            possible_meteorites[N] = [tracks[N][0], tracks[N][1], tracks[N][2]]
#             possible_meteorites.append([])
#             possible_meteorites[-1].append([tracks[N][0]])
#             possible_meteorites[-1].append([tracks[N][1]])
#             possible_meteorites[-1].append([tracks[N][2]])
    
    
    return tracks, possible_meteorites



In [ ]:

def get_next_neighbours(dict_x, dict_y, dict_z, width):
    nnn_x = []
    nnn_y = []
    nnn_z = []
    for k in range(len(dict_x)):
        x_x_cut = []
        y_x_cut = []
        z_x_cut = []
        x_y_cut = []
        y_y_cut = []
        z_y_cut = []
        x = dict_x[k]
        y = dict_y[k]
        z = dict_z[k]
        for i in range(len(dict_x)):
            if x-width*min_x_dif < dict_x[i] < x+width*min_x_dif:
                x_x_cut.append(dict_x[i])
                y_x_cut.append(dict_y[i])
                z_x_cut.append(dict_z[i])

        for i in range(len(x_x_cut)):
            if y-width*min_y_dif < y_x_cut[i] < y+width*min_y_dif:
                x_y_cut.append(x_x_cut[i])
                y_y_cut.append(y_x_cut[i])
                z_y_cut.append(z_x_cut[i])
        nnn_x.append(x_y_cut)
        nnn_y.append(y_y_cut)
        nnn_z.append(z_y_cut)
    return nnn_x, nnn_y, nnn_z
def check_z_neighbours(dict_x, dict_y, dict_z):
    nnn_x = []
    nnn_y = []
    nnn_z = []
    for i in range(len(dict_z)):
        close_times = np.concatenate((np.where(np.array(dict_z) == dict_z[i]-1)[0],
                                    np.where(np.array(dict_z) == dict_z[i])[0],
                                    np.where(np.array(dict_z) == dict_z[i]+1)[0]))
        nnn_x.append(np.array(dict_x)[close_times])
        nnn_y.append(np.array(dict_y)[close_times])
        nnn_z.append(np.array(dict_z)[close_times])
    return nnn_x, nnn_y, nnn_z

def cleaning_cut(dict_x,dict_y,dict_z, dict_brightness, width, length):
    x_reduced = []
    y_reduced = []
    z_reduced = []
    brightness_reduced = []
    nn_x, nn_y, nn_z = get_next_neighbours(dict_x, dict_y, dict_z, width)
    for k in range(len(dict_x)):
        if len(np.unique(nn_y[k]))>length:
            x_reduced.append(dict_x[k])
            y_reduced.append(dict_y[k])
            z_reduced.append(dict_z[k])
            brightness_reduced.append(dict_brightness[k])
    return x_reduced, y_reduced, z_reduced, brightness_reduced
def cleaning_z_cut(dict_x,dict_y,dict_z, width):
    nnn_x, nnn_y, nnn_z = get_next_neighbours(dict_x, dict_y, dict_z, width)
    for i in range(len(dict_z)):
        counter = 0
        for j in range(len(nnn_z[i])):
            if dict_z[i]-2 < nnn_z[i][j] < dict_z[i]+2:
                    counter = counter + 1
        if counter < 7: 
            print("less than 7 values in nn_z for index", i)
def sort_into_tracks(dict_x,dict_y,dict_z,dict_brightness, width):
    nnn_x, nnn_y, nnn_z = get_next_neighbours(dict_x, dict_y, dict_z, width)
    tracks = {}
    max_tracks = 9
    for N in range(1,max_tracks):
        tracks["x-"+str(N)] = []
        tracks["y-"+str(N)] = []
        tracks["z-"+str(N)] = []
        tracks["brightness-"+str(N)] = []
    k=0
    for i in range(len(dict_x)):
        appended_to_track = False
        empty_first_track = True
        if k == 0:
            tracks["x-"+str(1)].append(dict_x[i])
            tracks["y-"+str(1)].append(dict_y[i])
            tracks["z-"+str(1)].append(dict_z[i])
            tracks["brightness-"+str(1)].append(dict_brightness[i])
            appended_to_track  = True 
            k = k+1
            continue
        for N in range(1,max_tracks):
            if len(tracks["x-"+str(N)])==0:
                continue            
            if tracks["x-"+str(N)][-1] in nnn_x[i] and tracks["y-"+str(N)][-1] in nnn_y[i]:
                if dict_z[i] -2< tracks["z-"+str(N)][-1]:
                    tracks["x-"+str(N)].append(dict_x[i])
                    tracks["y-"+str(N)].append(dict_y[i])
                    tracks["z-"+str(N)].append(dict_z[i])
                    tracks["brightness-"+str(N)].append(dict_brightness[i])
                    appended_to_track  = True 
                    break
                else:
                    if len(tracks["x-"+str(N)])<3:
                        tracks["x-"+str(N)] = []
                        tracks["y-"+str(N)] = []
                        tracks["z-"+str(N)] = []
                        tracks["brightness-"+str(N)] = []
                        print("track", N, " was deleted at time", dict_z[i] )
                        
        if appended_to_track == False:
            for N in range(1,max_tracks):
                if len(tracks["x-"+str(N)])==0:
                    tracks["x-"+str(N)].append(dict_x[i])
                    tracks["y-"+str(N)].append(dict_y[i])
                    tracks["z-"+str(N)].append(dict_z[i])
                    tracks["brightness-"+str(N)].append(dict_brightness[i])
                    break
    possible_meteorites = {}
    for N in range(1,max_tracks):
        if len(tracks["x-"+str(N)]) != 0 and len(np.unique(tracks["z-"+str(N)]))<2:
            print("Track", N, "from", np.unique(tracks["z-"+str(N)]),
                  ",too fast", len(tracks["z-"+str(N)]), " entires, hence no track")
            possible_meteorites["x-"+str(N)] = tracks["x-"+str(N)] 
            possible_meteorites["y-"+str(N)] = tracks["y-"+str(N)] 
            possible_meteorites["z-"+str(N)] = tracks["z-"+str(N)] 
            possible_meteorites["brightness-"+str(N)] = tracks["brightness-"+str(N)]
            tracks["x-"+str(N)] = []
            tracks["y-"+str(N)] = []
            tracks["z-"+str(N)] = []
            tracks["brightness-"+str(N)] = []
        try:
            if possible_meteorites["z-"+str(N)]:
                pass # add tracks with same time
        except:
            pass
        
            
    return tracks, possible_meteorites



def sort_into_tracks_old(dict_x,dict_y,dict_z, width):
    nnn_x, nnn_y, nnn_z = get_next_neighbours(dict_x, dict_y, dict_z, width)
    tracks = {}
    for N in range(1,6):
        tracks["x-"+str(N)] = []
        tracks["y-"+str(N)] = []
        tracks["z-"+str(N)] = []
    k=0
    for i in range(len(dict_x)):
        appended_to_track = False
        empty_first_track = True
        if k == 0:
            tracks["x-"+str(1)].append(dict_x[i])
            tracks["y-"+str(1)].append(dict_y[i])
            tracks["z-"+str(1)].append(dict_z[i])
            appended_to_track  = True 
            k = k+1
            continue
        for N in range(1,6):
            if len(tracks["x-"+str(N)])==0:
                continue            
            if tracks["x-"+str(N)][-1] in nnn_x[i] and tracks["y-"+str(N)][-1] in nnn_y[i]:
                if dict_z[i] -2< tracks["z-"+str(N)][-1]:
                    tracks["x-"+str(N)].append(dict_x[i])
                    tracks["y-"+str(N)].append(dict_y[i])
                    tracks["z-"+str(N)].append(dict_z[i])
                    appended_to_track  = True 
                    break
                else:
                    if len(tracks["x-"+str(N)])<3:
                        tracks["x-"+str(N)] = []
                        tracks["y-"+str(N)] = []
                        tracks["z-"+str(N)] = []
                        print("track", N, " was deleted at time", dict_z[i] )
                        
        if appended_to_track == False:
            for N in range(1,6):
                if len(tracks["x-"+str(N)])==0:
                    tracks["x-"+str(N)].append(dict_x[i])
                    tracks["y-"+str(N)].append(dict_y[i])
                    tracks["z-"+str(N)].append(dict_z[i])
                    break
    for N in range(1,6):
        if len(tracks["x-"+str(N)]) != 0 and len(np.unique(tracks["z-"+str(N)]))<2:
            print("Track", N, "from", np.unique(tracks["z-"+str(N)]),
                  ",too fast", len(tracks["z-"+str(N)]), " entires, hence no track")
            tracks["x-"+str(N)] = []
            tracks["y-"+str(N)] = []
            tracks["z-"+str(N)] = []
            
    return tracks

def get_track_width(dict_x, dict_y):
    popt, pcov = curve_fit(lin_fit, dict_x, dict_y)
    d_from_lin_fit = []
    for i in range(len(dict_x)):
        x_p = dict_x[i]
        y_p = dict_y[i]
        t_2 = x_p/popt[0]+y_p
        x_s = popt[0]/(popt[0]**2+1)*(t_2 -popt[1])
        y_s = popt[0]*x_s+popt[1]
        d_from_lin_fit.append(np.sqrt(((x_p-x_s)/min_x_dif)**2+((y_p-y_s)/min_y_dif)**2))
        width = round(np.average(d_from_lin_fit)*2, 4)
        std_width = round(np.std(d_from_lin_fit)*2, 4)
    #print("Average distance from linfit: ",  round(np.average(d_from_lin_fit), 4))
    #print("Average width = ", round(np.average(d_from_lin_fit)*2, 4), "Pixels")
    return width, std_width
def draw_box(m, t):
    y_top = np.where(geom_hess5_yc_from_root == np.max(geom_hess5_yc_from_root))[0]
    y_bot = np.where(geom_hess5_yc_from_root == np.min(geom_hess5_yc_from_root))[0]
    x_top = np.where(geom_hess5_xc_from_root == np.max(geom_hess5_xc_from_root))[0]
    x_bot = np.where(geom_hess5_xc_from_root == np.min(geom_hess5_xc_from_root))[0]

    #draw a box 
    box_x = []
    box_y = []
    box_x.append(sorted(geom_hess5_xc_from_root[y_top])[0])
    box_x.append(sorted(geom_hess5_xc_from_root[y_top])[-1])
    box_x.append(sorted(geom_hess5_xc_from_root[x_top])[0])
    box_x.append(sorted(geom_hess5_xc_from_root[y_bot])[-1])
    box_x.append(sorted(geom_hess5_xc_from_root[y_bot])[0])
    box_x.append(sorted(geom_hess5_xc_from_root[x_bot])[0])
    box_x.append(sorted(geom_hess5_xc_from_root[x_bot])[1])
    box_x.append(sorted(geom_hess5_xc_from_root[y_top])[0])

    box_y.append(sorted(geom_hess5_yc_from_root[y_top])[0])
    box_y.append(sorted(geom_hess5_yc_from_root[y_top])[-1])
    box_y.append(sorted(geom_hess5_yc_from_root[x_top])[0])
    box_y.append(sorted(geom_hess5_yc_from_root[y_bot])[-1])
    box_y.append(sorted(geom_hess5_yc_from_root[y_bot])[0])
    box_y.append(sorted(geom_hess5_yc_from_root[x_bot])[0])
    box_y.append(sorted(geom_hess5_yc_from_root[x_bot])[1])
    box_y.append(sorted(geom_hess5_yc_from_root[y_top])[0])
    # 7 lines to draw the box:    
    plt.plot(box_x, box_y, c = "blue")
    #plt.plot((box_x[-1], box_x[0]),(box_y[-1], box_y[0]), c = "blue")
    #find intersections with lin_fit:
    box_lines_param = {}
    for i in range(0,7):
        if (box_x[i+1]-box_x[i]) !=0:
            m_box = (box_y[i+1]-box_y[i])/(box_x[i+1]-box_x[i])
            t_box = box_y[i]- m_box*box_x[i]
            box_lines_param[i] =[m_box, t_box] 
        else:
            box_lines_param[i] = [box_x[i]]
#     m_box = (box_y[0]-box_y[-1])/(box_x[0]-box_x[-1])
#     t_box = box_y[-1]- m_box*box_x[-1]
#     box_lines_param[6] =[m_box, t_box] 
    intersections = []
    for i in range(len(box_lines_param)):
        if len(box_lines_param[i])==1:
            pass
        else: 
            x_sp = (box_lines_param[i][1] - t)/(m - box_lines_param[i][0])
        y_sp = m * x_sp + t
        if -1.25< x_sp < 1.25:
            if -1.25< y_sp < 1.25:
                intersections.append([x_sp, y_sp])
    intersections = np.array(intersections)
    plt.scatter(intersections[:,0],intersections[:,1], marker ="o", facecolors='none', edgecolors='black', s = 100)
    print("Coordinates of intersections:", np.around(intersections[0], 4), ", ", np.around(intersections[1], 4))
    return(intersections)
    

In [ ]:
def trail_finder(dict_pix, dict_tmp, dict_brightness):   
    tracks = {}
    possible_meteorites = {}
    if (len(dict_pix)==0):
        print("Too few values in run", nrun)
        return tracks, possible_meteorites
#     x = []
#     y = []
#     for i in range(len(pix)):
#         x.append(flash_geom_x[pix[i]])
#         y.append(flash_geom_y[pix[i]])##there must be a better way...
    pix = []
    tmp = []
    brightness = []
    for i in range(len(dict_pix)):
        mask_time = np.logical_and( dict_tmp>dict_tmp[i]-5,dict_tmp<dict_tmp[i]+5)
        st = set(nn_pix[int(dict_pix[i])])
        mask_space = [i for i, e in enumerate(dict_pix[mask_time]) if e in st]
        
        if len(set(dict_pix[mask_time][mask_space]))>0:
            pix.append(dict_pix[i])
            tmp.append(dict_tmp[i])
            brightness.append(dict_brightness[i])
    tracks[1] = {}
    tracks[1]["pix"] = [pix[0]]
    tracks[1]["tmp"] = [tmp[0]]
    tracks[1]["brightness"] = [brightness[0]]
    N_tot = 1
    print(len(pix))
    for i in range(1, len(pix)):
        appended_to_track = False
        for N in tracks:
            if len(set(tracks[N]["pix"][-20:]) & set(nn_pix[pix[i]]))>0:
                if tmp[i]-5 < tracks[N]["tmp"][-1]:
                    tracks[N]["pix"].append(pix[i])
                    tracks[N]["tmp"].append(tmp[i])
                    tracks[N]["brightness"].append(brightness[i])
                    appended_to_track  = True 
                    break
#             else:
#                 if len(tracks[N]["tmp"])<2:
#                     tracks.pop(N, None)
        if appended_to_track == False:
            N_tot+=1
            print(N_tot)
            tracks[N_tot] = {}
            tracks[N_tot]["pix"] = [pix[i]]
            tracks[N_tot]["tmp"] = [tmp[i]]
            tracks[N_tot]["brightness"] = [brightness[i]]
            continue
    N_track = 1
    N_to_pop = []
    for N in list(tracks.keys()):
        if len(tracks[N]["pix"]) < 3:
            tracks.pop(N, None)
            continue
    for N in tracks:
#         if len(tracks[N]["pix"]) == 0:
#             tracks.pop(N, None)
#             continue
        if len(set(tracks[N]["pix"]))<3:
            print("Track", N, "from", set(tracks[N]["tmp"]),
                  ",too few pixels", len(tracks[N]["pix"]))
            possible_meteorites[N_track] = {}
            possible_meteorites[N_track]["pix"] = tracks[N]["pix"] 
            possible_meteorites[N_track]["tmp"] = tracks[N]["tmp"] 
            possible_meteorites[N_track]["brightness"] = tracks[N]["brightness"]
            N_to_pop.append(N)#tracks.pop(N, None)
            N_track+=1
        elif len(set(tracks[N]["tmp"]))<3:
            print("Track", N, "from", set(tracks[N]["tmp"]),
                  ",too few different times", set(tracks[N]["tmp"]))
            possible_meteorites[N_track] = {}
            possible_meteorites[N_track]["pix"] = tracks[N]["pix"] 
            possible_meteorites[N_track]["tmp"] = tracks[N]["tmp"] 
            possible_meteorites[N_track]["brightness"] = tracks[N]["brightness"]
            N_to_pop.append(N)#tracks.pop(N, None)
            N_track+=1
#     for N in N_to_pop:
#         tracks.pop(N, None)
    
    
    return tracks, possible_meteorites
def get_neighbour_pixel_entries(dict_x,dict_y,dict_z, dict_pix, dict_brightness):
    #check für jeden Eintrag, ob in den next_neighbours einträge innerhalb der nächsten 2s liegen
    #nn_pix ist array mit array von neighbours und next neigbours als einträgen (2d-array), nn_pix[N] gibt nn von pixId N an
    min_len = 0
    x = []
    y = []
    z = []
    pix = []
    brightness = []
    tracks = {}
    for i in range(len(dict_pix)):
        mask_time = np.logical_and( dict_z>dict_z[i]-3,dict_z<dict_z[i]+3)
        st = set(nn_pix[int(dict_pix[i])])
        mask_space = [i for i, e in enumerate(dict_pix[mask_time]) if e in st]
        
        if len(np.unique(dict_pix[mask_time][mask_space]))>min_len:
            x.append(dict_x[i])
            y.append(dict_y[i])
            z.append(dict_z[i])
            pix.append(dict_pix[i])
            brightness.append(dict_brightness[i])
    x = np.array(x)
    y = np.array(y)
    z = np.array(z)
    pix = np.array(pix)
    brightness = np.array(brightness)
    
    time_values, time_counts = np.unique(z, return_counts=True)
    if len(z)>0:  
        mask_min_counts = time_counts>0
        set_min_counts =set(time_values[mask_min_counts])
        mask = [i for i, e in enumerate(z) if e in set_min_counts] 
        
        print(len(mask), len(z))
        # mask is for all values in the previously appended arrays to get only the values which also fulfill mask_min_counts 
        plot_scatter(x[mask],y[mask],z[mask])
        draw_box()
        plt.show()
    else:
        print("No suitable values found for", nrun)
        return
    if (len(x[mask])==0):
        print("Too few values in run", nrun)
        return
    
    
    max_tracks = 100 #stupid, but if neccessary just add a 0
    for N in range(1,max_tracks):
        tracks[N] = {}
        tracks[N]["x"] = []
        tracks[N]["y"] = []
        tracks[N]["z"] = []
        tracks[N]["pix"] = []
        tracks[N]["brightness"] = []
    for i in range(len(x[mask])):
        appended_to_track = False
        if i == 0:
            tracks[1]["x"].append(x[mask][i])
            tracks[1]["y"].append(y[mask][i])
            tracks[1]["z"].append(z[mask][i])
            tracks[1]["pix"].append(pix[mask][i])
            tracks[1]["brightness"].append(brightness[mask][i])
            appended_to_track  = True 
            continue
        for N in range(1,max_tracks):
            if N not in tracks:
                continue       
            if len(tracks[N]["pix"])==0:
                continue
            if tracks[N]["pix"][-1] in nn_pix[int(pix[mask][i])]:
                if z[mask][i] -2< tracks[N]["z"][-1]:
                    tracks[N]["x"].append(x[mask][i])
                    tracks[N]["y"].append(y[mask][i])
                    tracks[N]["z"].append(z[mask][i])
                    tracks[N]["pix"].append(pix[mask][i])
                    tracks[N]["brightness"].append(brightness[mask][i])
                    appended_to_track  = True 
                    break
            else:
                if len(tracks[N]["x"])<2:
                    tracks.pop(N, None)
          
                        
        if appended_to_track == False:
            for N in range(1,max_tracks):
                if N not in tracks:
                    continue
                if len(tracks[N]["x"])==0:
                    tracks[N]["x"].append(x[mask][i])
                    tracks[N]["y"].append(y[mask][i])
                    tracks[N]["z"].append(z[mask][i])
                    tracks[N]["pix"].append(pix[mask][i])
                    tracks[N]["brightness"].append(brightness[mask][i])
                    break
               
                        
    possible_meteorites = {}
    N_track = 1
    for N in range(1,max_tracks):
        if N not in tracks:
            continue
        if len(tracks[N]["x"]) == 0:
            tracks.pop(N, None)
            continue
        possible_meteorites[N_track] = {}
        #if len(tracks[N]["x"]) != 0 and len(np.unique(tracks[N]["z"]))<3:
        if len(np.unique(tracks[N]["pix"]))<3:
            print("Track", N, "from", np.unique(tracks[N]["z"]),
                  ",too few pixels", len(tracks[N]["pix"]))
            possible_meteorites[N_track]["x"] = tracks[N]["x"] 
            possible_meteorites[N_track]["y"] = tracks[N]["y"] 
            possible_meteorites[N_track]["z"] = tracks[N]["z"] 
            possible_meteorites[N_track]["pix"] = tracks[N]["pix"] 
            possible_meteorites[N_track]["brightness"] = tracks[N]["brightness"]
            tracks.pop(N, None)
        elif len(np.unique(tracks[N]["z"]))<3:
            print("Track", N, "from", np.unique(tracks[N]["z"]),
                  ",too few different times", set(tracks[N]["z"]))
            possible_meteorites[N_track]["x"] = tracks[N]["x"] 
            possible_meteorites[N_track]["y"] = tracks[N]["y"] 
            possible_meteorites[N_track]["z"] = tracks[N]["z"] 
            possible_meteorites[N_track]["pix"] = tracks[N]["pix"] 
            possible_meteorites[N_track]["brightness"] = tracks[N]["brightness"]
            tracks.pop(N, None)
            
        try:
            if possible_meteorites[N_track]["z"]:
                pass # add tracks with same time
        except:
            if len(np.unique(tracks[N]["pix"]))>2:
                plot_scatter(tracks[N]["x"], tracks[N]["y"], tracks[N]["z"])
                draw_box_new(0,0)
                plt.show()
        N_track+=1
#         for N in reversed(range(1,max_tracks)):
#             try:
#                 #print(N, np.unique(possible_meteorites[N]["z"]))
#                 if len(set(np.unique(possible_meteorites[N]["z"]) + np.min(possible_meteorites[N]["z"])-1) & 
#                        set(np.unique(possible_meteorites[N-1]["z"]) + np.max(possible_meteorites[N-1]["z"])+1 ))>0:
#                     print("yes")
#                     possible_meteorites[N-1]["x"] = possible_meteorites[N-1]["x"]+possible_meteorites[N]["x"]
#                     possible_meteorites[N-1]["y"] = possible_meteorites[N-1]["y"]+possible_meteorites[N]["y"]
#                     possible_meteorites[N-1]["z"] = possible_meteorites[N-1]["z"]+possible_meteorites[N]["z"]
#                     possible_meteorites[N-1]["pix"] = possible_meteorites[N-1]["pix"]+possible_meteorites[N]["pix"]
#                     possible_meteorites[N-1]["brightness"] = possible_meteorites[N-1]["brightness"]+possible_meteorites[N]["brightness"]
#                     possible_meteorites.pop(N, None)

#                 else:
#                     if len(np.unique(possible_meteorites[N]["pix"]))<5:
#                         possible_meteorites.pop(N, None)
#             except:
#                 pass
        
            
    return tracks, possible_meteorites
    
   


In [ ]:
#Define camera geometry:
fin_data = open_file("D:/Masterarbeit ECAP/2022-04-06/Thomas/gamma_sat_postselect.h5", mode="r")
geom_hess1_xc_from_root = np.squeeze(fin_data.get_node('/configuration/instrument/telescope/camera/geometry_0').col('pix_x'))
geom_hess1_yc_from_root = np.squeeze(fin_data.get_node('/configuration/instrument/telescope/camera/geometry_0').col('pix_y'))
geom_hess5_xc_from_root = np.squeeze(fin_data.get_node('/configuration/instrument/telescope/camera/geometry_1').col('pix_x'))
geom_hess5_yc_from_root = np.squeeze(fin_data.get_node('/configuration/instrument/telescope/camera/geometry_1').col('pix_y'))
a = sorted(geom_hess5_xc_from_root)
b = sorted(geom_hess5_yc_from_root)
for i in range(len(a)-1):
    if a[i+1]-a[i]>0:
        min_x_dif = round(a[i+1]-a[i], 5)
        print(min_x_dif)
        break
for i in range(len(b)-1):
    if b[i+1]-b[i]>0:
        min_y_dif = round(b[i+1]-b[i], 5)
        print(min_y_dif)
        break